In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import scipy.stats as ss
from scipy.stats.mstats import winsorize
from matplotlib import pyplot as plt
import statsmodels.formula.api as smf
import statsmodels.api as sm

# 新三板

In [128]:
# path = 'C:/Users/84678/Desktop/做市商效率/数据处理/新三板企业.xlsx'
# df_n = pd.read_excel(path)

In [129]:
# df_n_c = df_n.loc[df_n.day.isnull().values == True]

In [130]:
# df_n1 = df_n.dropna()
# df_n_m = df_n.loc[df_n.day < dt.datetime(2017,1,1,0,0,0)]

In [131]:
# df_n_c.to_excel("C:/Users/84678/Desktop/做市商效率/数据处理/新三板竞价企业.xlsx" , index = False)
# df_n_m.to_excel("C:/Users/84678/Desktop/做市商效率/数据处理/新三板做市企业.xlsx" , index = False)

In [41]:
#导入做市交易企业数据
path_m = 'C:/Users/84678/Desktop/做市商效率/数据处理/新三板做市企业.xlsx'
df_n_m_len = pd.read_excel(path_m,sheet_name = "数据剔除后（时间）",index_col = 0)
df_n_m_condition = pd.read_excel(path_m, sheet_name = "数据剔除（区间交易状态）",index_col = 0)

In [42]:
list_index = []
for i in range(df_n_m_len.shape[0]):
    r = df_n_m_len.iloc[i]
    r1 = df_n_m_condition.iloc[i].to_frame().T
    date_b1 = r['做市前200交易日']
    date_a2 = r['做市后200交易日']
    r_temp = r1.loc[:,date_b1:date_a2]
    
    r_temp = r_temp[~r_temp.isin([0,'停牌一天'])].dropna(axis=0)
    list_index.extend(r_temp.index.values)

In [43]:
list_index = list(filter(None,list_index))

In [44]:
df_n_m = pd.read_excel(path_m,sheet_name = "数据表",index_col = 0)
df_n_m 

,名称,做市首日,做市前200交易日,做市前100交易日,做市后200交易日,收盘价,交易天数,流通股合计,匹配期,总资产,做市交易前一日
代码,,,,,,,,,,,
430014.NQ,恒业世纪,2015/11/24,2015/1/27,2015/6/26,2016/9/13,3.581200,54,63218421.0,2015/6/30,2.218597e+08,2015/11/23
430021.NQ,海鑫科金,2016/1/29,2015/4/10,2015/9/1,2016/11/28,21.406700,68,135298934.0,2015/12/31,1.169655e+09,2016/1/28
430022.NQ,五岳鑫,2015/12/8,2015/2/10,2015/7/10,2016/9/29,3.002300,10,20661000.0,2015/6/30,6.642912e+07,2015/12/7
430027.NQ,北科光大,2016/3/28,2015/6/2,2015/10/30,2017/1/18,8.528400,40,21249986.0,2015/12/31,1.394838e+08,2016/3/25
430033.NQ,彩讯科技,2014/8/25,2013/11/4,2014/4/1,2015/6/23,1.842800,30,23664109.0,2014/6/30,1.615971e+08,2014/8/22
...,...,...,...,...,...,...,...,...,...,...,...
836594.NQ,摘牌逸洁(退市),2016/12/12,2016/2/19,2016/7/14,2017/10/10,0.000000,0,0.0,2016/6/30,1.235483e+08,2016/12/9
836676.NQ,佳化股份(退市),2016/12/14,2016/2/23,2016/7/18,2017/10/12,3.857500,2,0.0,2016/6/30,1.798171e+08,2016/12/13
836694.NQ,摘牌家电(退市),2016/11/10,2016/1/13,2016/6/14,2017/9/1,3.826429,9,0.0,2016/6/30,2.101048e+08,2016/11/9


In [45]:
df_n_m = df_n_m.loc[list_index]

In [46]:
df_n_m = df_n_m.drop(df_n_m.loc[(df_n_m['收盘价']==0) & (df_n_m['交易天数']==0) & (df_n_m['流通股合计'] ==0)].index)
df_n_m = df_n_m.drop(df_n_m.loc[(df_n_m['总资产']==0)].index)
df_n_m

,名称,做市首日,做市前200交易日,做市前100交易日,做市后200交易日,收盘价,交易天数,流通股合计,匹配期,总资产,做市交易前一日
代码,,,,,,,,,,,
430014.NQ,恒业世纪,2015/11/24,2015/1/27,2015/6/26,2016/9/13,3.5812,54,63218421.0,2015/6/30,2.218597e+08,2015/11/23
430022.NQ,五岳鑫,2015/12/8,2015/2/10,2015/7/10,2016/9/29,3.0023,10,20661000.0,2015/6/30,6.642912e+07,2015/12/7
430027.NQ,北科光大,2016/3/28,2015/6/2,2015/10/30,2017/1/18,8.5284,40,21249986.0,2015/12/31,1.394838e+08,2016/3/25
430033.NQ,彩讯科技,2014/8/25,2013/11/4,2014/4/1,2015/6/23,1.8428,30,23664109.0,2014/6/30,1.615971e+08,2014/8/22
430037.NQ,联飞翔,2014/8/25,2013/11/4,2014/4/1,2015/6/23,3.2805,62,77348622.0,2014/6/30,3.024244e+08,2014/8/22
...,...,...,...,...,...,...,...,...,...,...,...
834180.NQ,摘牌鸿立(退市),2016/9/8,2015/11/19,2016/4/18,2017/7/11,4.1770,3,0.0,2016/6/30,4.264862e+07,2016/9/7
834262.NQ,康富科技(退市),2016/10/10,2015/12/10,2016/5/10,2017/8/1,2.8000,2,0.0,2016/6/30,1.421099e+08,2016/9/30
834509.NQ,凯迪威(退市),2016/10/10,2015/12/10,2016/5/10,2017/8/1,10.1400,4,15626666.0,2016/6/30,1.502929e+08,2016/9/30


In [47]:
#定义函数，用于给四个变量分组
def divide(df):
    bins = 4
    bins1 = bins - 1
    
    try:
        df['Segment_a'] = pd.DataFrame(bins - pd.qcut(df['总资产'], bins).cat.codes)
    except:
        df['Segment_a'] = 0
        df['Segment_a'].loc[df['总资产'] != 0] = bins1 - pd.qcut(df['总资产'].loc[df['总资产'] != 0],bins1).cat.codes
        df['Segment_a'][df['总资产'] == 0] = 0
        df['Segment_a'] += 1
        df['Segment_a'] = df['Segment_a'].astype(int)
    
    try:
        df['Segment_c'] = bins - pd.qcut(df['收盘价'], bins).cat.codes
    except:
        df['Segment_c'] = 0
        df['Segment_c'].loc[df['收盘价'] != 0] = bins1 - pd.qcut(df['收盘价'].loc[df['收盘价'] != 0],bins1).cat.codes
        df['Segment_c'][df['收盘价'] == 0] = 0
        df['Segment_c'] += 1
        df['Segment_c'] = df['Segment_c'].astype(int)
    
    try:
        df['Segment_d'] = bins - pd.qcut(df['交易天数'], bins).cat.codes
    except:
        df['Segment_d'] = 0
        df['Segment_d'].loc[df['交易天数'] != 0] = bins1 - pd.qcut(df['交易天数'].loc[df['交易天数'] != 0],bins1).cat.codes
        df['Segment_d'][df['交易天数'] == 0] = 0
        df['Segment_d'] += 1
        df['Segment_d'] = df['Segment_d'].astype(int)

            
    #     num = df.shape[0]
    #     zero, hund = (df['交易天数'] == 0).sum(), (df['交易天数'] == 100).sum()
    #     # print(f'num:{num}, zero:{zero}, hund:{hund}')
    #     if 3*zero>=num:
    #         if 2*hund>=(num-zero):
    #             df['Segment_d'] = 2
    #             df['Segment_d'][df['交易天数'] == 0] = 3
    #             df['Segment_d'][df['交易天数'] == 100] = 1
    #         else:
    #             df['Segment_d'] = 3
    #             df['Segment_d'] = 2 - pd.qcut(df['交易天数'].loc[df['交易天数'] != 0],2).cat.codes
    #             df['Segment_d'][df['交易天数'] == 0] = 3
    #     elif 3*hund>=num:
    #         if 2*zero>=(num-hund):
    #             df['Segment_d'] = 2
    #             df['Segment_d'][df['交易天数'] == 0] = 3
    #             df['Segment_d'][df['交易天数'] == 100] = 1
    #         else:
    #             df['Segment_d'] = 1
    #             df['Segment_d'] = 3 - pd.qcut(df['交易天数'].loc[df['交易天数'] != 100],2).cat.codes
    #             df['Segment_d'][df['交易天数'] == 0] = 1
    #     df['Segment_d'] = df['Segment_d'].astype(int)
                
        
    try:
        df['Segment_s'] = bins - pd.qcut(df['流通股合计'],bins).cat.codes
    except:
        df['Segment_s'] = 0
        df['Segment_s'].loc[df['流通股合计'] != 0] = bins1 - pd.qcut(df['流通股合计'].loc[df['流通股合计'] != 0],bins1).cat.codes
        df['Segment_s'].loc[df['流通股合计'] == 0] = 0
        df['Segment_c'] += 1
        df['Segment_s'] = df['Segment_s'].astype(int)

    return df

In [48]:
divide(df_n_m)

,名称,做市首日,做市前200交易日,做市前100交易日,做市后200交易日,收盘价,交易天数,流通股合计,匹配期,总资产,做市交易前一日,Segment_a,Segment_c,Segment_d,Segment_s
代码,,,,,,,,,,,,,,,
430014.NQ,恒业世纪,2015/11/24,2015/1/27,2015/6/26,2016/9/13,3.5812,54,63218421.0,2015/6/30,2.218597e+08,2015/11/23,1,3,1,1
430022.NQ,五岳鑫,2015/12/8,2015/2/10,2015/7/10,2016/9/29,3.0023,10,20661000.0,2015/6/30,6.642912e+07,2015/12/7,3,3,2,2
430027.NQ,北科光大,2016/3/28,2015/6/2,2015/10/30,2017/1/18,8.5284,40,21249986.0,2015/12/31,1.394838e+08,2016/3/25,2,2,1,2
430033.NQ,彩讯科技,2014/8/25,2013/11/4,2014/4/1,2015/6/23,1.8428,30,23664109.0,2014/6/30,1.615971e+08,2014/8/22,2,4,1,1
430037.NQ,联飞翔,2014/8/25,2013/11/4,2014/4/1,2015/6/23,3.2805,62,77348622.0,2014/6/30,3.024244e+08,2014/8/22,1,3,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
834180.NQ,摘牌鸿立(退市),2016/9/8,2015/11/19,2016/4/18,2017/7/11,4.1770,3,0.0,2016/6/30,4.264862e+07,2016/9/7,4,3,3,4
834262.NQ,康富科技(退市),2016/10/10,2015/12/10,2016/5/10,2017/8/1,2.8000,2,0.0,2016/6/30,1.421099e+08,2016/9/30,2,3,4,4
834509.NQ,凯迪威(退市),2016/10/10,2015/12/10,2016/5/10,2017/8/1,10.1400,4,15626666.0,2016/6/30,1.502929e+08,2016/9/30,2,1,3,2


In [173]:
df_n_m.to_excel("C:/Users/84678/Desktop/做市商效率/数据处理/新三板做市企业（处理后）.xlsx" )

In [38]:
#导入竞价交易企业数据
path_c = 'C:/Users/84678/Desktop/做市商效率/数据处理/新三板竞价企业.xlsx'
df_n_c_assets = pd.read_excel(path_c, sheet_name = "总资产",index_col = 0)
df_n_c_close = pd.read_excel(path_c, sheet_name = "收盘价",index_col = 0)
df_n_c_tradedays = pd.read_excel(path_c, sheet_name = "交易天数",index_col = 0)
df_n_c_share = pd.read_excel(path_c, sheet_name = "流通股合计",index_col = 0)
df_n_c_condition = pd.read_excel(path_c, sheet_name = "交易状态",index_col = 0)

In [49]:
df_compare = pd.DataFrame()

#排除做市日前后200天内有停牌的公司
for i in range(df_n_m.shape[0]):
    r = df_n_m.iloc[i]
    date_b1 = r['做市前200交易日']
    date_b2 = r['做市前100交易日']
    date_b3 = r['做市交易前一日']
    date_a1 = r['做市首日']
    date_a2 = r['做市后200交易日']
    date_c = r['匹配期']
    temp = df_n_c_condition.loc[:,date_b1:date_a2]
    temp = temp.dropna(how = 'any')
    
    for j in range(temp.shape[0]):
        if '停牌一天' in temp.iloc[j]:
            temp = temp.drop[j]
            
    #对每个做市交易的公司只选择temp内的竞价交易的企业
    list_index = temp.index.values
    df_n_c_assets1 = df_n_c_assets.loc[list_index]
    df_n_c_close1 = df_n_c_close.loc[list_index] 
    df_n_c_tradedays1 = df_n_c_tradedays.loc[list_index] 
    df_n_c_share1 = df_n_c_share.loc[list_index]
    
    #计算区间内平均收盘价，交易天数和匹配期的总资产和流通股合计
    close = (df_n_c_close1.loc[ : , date_b2:date_b3]).mean(1)
    tradedays_temp = (df_n_c_tradedays1.loc[:,date_b2:date_b3]).fillna(0)
    tradedays = tradedays_temp.gt(0).sum(axis=1)
    assets = df_n_c_assets1.loc[ : , date_c]
    share = df_n_c_share1.loc[ : , date_c]
    
    #将竞价企业数据集合在一个表中
    df_n_c = pd.concat([assets,close,tradedays,share],axis = 1,ignore_index = True)
    df_n_c.columns = ['总资产','收盘价','交易天数','流通股合计']
    
    #删除收盘价、交易天数和流通股合计全为0的企业和总资产为0的企业后，对竞价交易的企业进行数据分组
    df_n_c = df_n_c.drop(df_n_c.loc[(df_n_c['总资产']==0)].index)
    divide(df_n_c)
    
    #匹配组别相同的数据
    df_n_c.reset_index(inplace = True)
    divi = pd.merge(df_n_c, pd.DataFrame(r).T , on=['Segment_a','Segment_c','Segment_d','Segment_s',], how="right")
    divi.dropna(inplace=True,how='any')
    
    #如果匹配公司>1，则选择总资产差异最小的公司
    if divi.shape[0] > 1 :
        compare = pd.DataFrame()
        compare_temp = pd.DataFrame()
        divi['diff_a'] = abs(divi['总资产_x'] - divi['总资产_y'])
        divi.sort_values(by=['diff_a'],inplace=True,ascending=True)
        compare = divi.iloc[0,:].to_frame().T
        compare_temp['treatment index'] =  pd.DataFrame(r).T.index
        compare_temp['control index'] = compare['代码'].values
        df_compare = pd.concat([df_compare,compare_temp] , axis = 0)

    #如果匹配公司=1，则选择唯一匹配的公司
    elif divi.shape[0] == 1:
        compare = pd.DataFrame()
        compare_temp = pd.DataFrame()
        compare = divi.iloc[0,:].to_frame().T
        compare_temp['treatment index'] =  pd.DataFrame(r).T.index
        compare_temp['control index'] = compare['代码'].values
        df_compare = pd.concat([df_compare,compare_temp] , axis = 0)

    else:
        compare = pd.DataFrame()
        compare_temp = pd.DataFrame()
        divi_temp = df_n_c
        divi_temp['diff_a'] = abs(df_n_c['总资产'] - pd.DataFrame(r).T['总资产'].values)
        divi_temp['diff_c'] = abs(df_n_c['收盘价'] - pd.DataFrame(r).T['收盘价'].values)
        divi_temp['diff_d'] = abs(df_n_c['交易天数'] - pd.DataFrame(r).T['交易天数'].values)
        divi_temp['diff_s'] = abs(df_n_c['流通股合计'] - pd.DataFrame(r).T['流通股合计'].values)
        divi_temp.sort_values(by=['diff_a'],inplace=True,ascending=True)
        divi_temp.reset_index(inplace = True)
        divi_temp.reset_index(inplace = True)
        divi_temp['rank_a'] = divi_temp['level_0']
        divi_temp = divi_temp.drop('level_0',axis=1)

        divi_temp.sort_values(by=['diff_c'],inplace=True,ascending=True)
        divi_temp.reset_index(inplace = True)
        divi_temp = divi_temp.drop('level_0',axis=1)
        divi_temp.reset_index(inplace = True)
        divi_temp['rank_c'] = divi_temp['level_0']
        divi_temp = divi_temp.drop('level_0',axis=1)

        divi_temp.sort_values(by=['diff_d'],inplace=True,ascending=True)
        divi_temp.reset_index(inplace = True)
        divi_temp = divi_temp.drop('level_0',axis=1)
        divi_temp.reset_index(inplace = True)
        divi_temp['rank_d'] = divi_temp['level_0']
        divi_temp = divi_temp.drop('level_0',axis=1)

        divi_temp.sort_values(by=['diff_s'],inplace=True,ascending=True)
        divi_temp.reset_index(inplace = True)
        divi_temp = divi_temp.drop('level_0',axis=1)
        divi_temp.reset_index(inplace = True)
        divi_temp['rank_s'] = divi_temp['level_0']
        divi_temp = divi_temp.drop('level_0',axis=1)

        divi_temp['rank'] = divi_temp['rank_a']+divi_temp['rank_c']+divi_temp['rank_d']+divi_temp['rank_s']
        divi_temp.sort_values(by=['rank'],inplace=True,ascending=True)

        compare = divi_temp.iloc[0,:].to_frame().T
        compare_temp['treatment index'] =  pd.DataFrame(r).T.index
        compare_temp['control index'] = compare['代码'].values
        df_compare = pd.concat([df_compare,compare_temp] , axis = 0)
        continue
    

C:\Users\84678\AppData\Local\Temp\ipykernel_28276\2765408281.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Segment_c'].loc[df['收盘价'] != 0] = bins1 - pd.qcut(df['收盘价'].loc[df['收盘价'] != 0],bins1).cat.codes
C:\Users\84678\AppData\Local\Temp\ipykernel_28276\2765408281.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Segment_c'][df['收盘价'] == 0] = 0
C:\Users\84678\AppData\Local\Temp\ipykernel_28276\2765408281.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

In [51]:
df_compare

,treatment index,control index
0,430014.NQ,430626.NQ
0,430022.NQ,430269.NQ
0,430027.NQ,430019.NQ
0,430033.NQ,430011.NQ
0,430037.NQ,430025.NQ
...,...,...
0,834180.NQ,834183.NQ
0,834262.NQ,834523.NQ
0,834509.NQ,833841.NQ
0,834716.NQ,835085.NQ


In [50]:
# 导出df_compare2
df_compare.to_excel("C:/Users/84678/Desktop/做市商效率/数据处理/compare2.xlsx" , index = False)

## 特征比较

In [52]:
# 导入处理组和控制组结果数据
n_treatment = pd.read_excel( 'C:/Users/84678/Desktop/做市商效率/流动性/两组数据对比.xlsx',sheet_name = "treatment",index_col = 0)
n_control = pd.read_excel( 'C:/Users/84678/Desktop/做市商效率/流动性/两组数据对比.xlsx',sheet_name = "control",index_col = 0)

In [53]:
df_treatment = n_treatment.iloc[:,4:]
df_treatment.describe()

,meanprice,outstanding,leverage,volatility,asset,roa
count,500.000000,5.000000e+02,500.000000,500.000000,5.000000e+02,500.000000
mean,7.657927,2.027208e+07,39.408506,307.917561,2.265519e+08,8.043267
std,8.602733,4.483485e+07,19.077143,2258.919490,5.132933e+08,9.414101
min,0.000000,0.000000e+00,0.686500,0.000000,6.601623e+06,-39.039000
25%,2.702203,4.270354e+06,24.921825,0.000000,6.276357e+07,3.140550
50%,5.204510,1.033421e+07,39.602350,8.049934,1.197801e+08,6.304700
75%,9.252781,2.125416e+07,52.510650,27.921732,2.178758e+08,11.483125
max,79.302300,6.215000e+08,93.713600,30120.983323,7.113536e+09,69.053500


In [54]:
df_control = n_control.iloc[:,4:]
df_control.describe()

,meanprice,outstanding,leverage,volatility,asset,roa
count,500.000000,5.000000e+02,500.000000,500.000000,5.000000e+02,500.000000
mean,7.399508,1.520070e+07,43.554065,60.100698,1.621978e+08,4.066788
std,8.227302,2.564438e+07,37.278263,519.837884,2.702100e+08,11.505241
min,0.000000,0.000000e+00,0.507900,0.000000,9.263622e+06,-49.923700
25%,2.000000,3.083312e+06,23.502625,0.000000,4.521013e+07,0.072775
50%,5.626950,7.750000e+06,39.487200,2.685714,8.033558e+07,3.482900
75%,9.361775,1.888179e+07,59.255375,12.305785,1.604497e+08,7.286975
max,71.246250,2.284767e+08,509.175400,9990.875808,2.521476e+09,153.965800


In [55]:
# 比较两个平均数的差异是否显著。当组别>2时，使用anova;否则可以直接使用t检验。

In [56]:
for i in range(6):    
    #print(ss.mannwhitneyu(df_treatment.iloc[:,i],df_control.iloc[:,i]))
    print(ss.ttest_ind(df_treatment.iloc[:,i],df_control.iloc[:,i]))

Ttest_indResult(statistic=0.48543552454684114, pvalue=0.627473984533298)
Ttest_indResult(statistic=2.195505428116291, pvalue=0.028356964453495573)
Ttest_indResult(statistic=-2.2136149376915015, pvalue=0.027080781242866964)
Ttest_indResult(statistic=2.3906142789748097, pvalue=0.01700476178967085)
Ttest_indResult(statistic=2.480732241321648, pvalue=0.013275175129590951)
Ttest_indResult(statistic=5.981245601861112, pvalue=3.0818309665703854e-09)


## 流动性

In [57]:
#非零交易天数和换手率
nonzero_t = pd.read_excel( 'C:/Users/84678/Desktop/做市商效率/流动性/流动性.xlsx',sheet_name = "nonzero_t",index_col = 0)
nonzero_c = pd.read_excel( 'C:/Users/84678/Desktop/做市商效率//流动性/流动性.xlsx',sheet_name = "nonzero_c",index_col = 0)
#成交量
volume_t = pd.read_excel( 'C:/Users/84678/Desktop/做市商效率/流动性/流动性.xlsx',sheet_name = "volume_t",index_col = 0)
volume_c = pd.read_excel( 'C:/Users/84678/Desktop/做市商效率/流动性/流动性.xlsx',sheet_name = "volume_c",index_col = 0)
#价格冲击指标 Hmihud
amihud_t = pd.read_excel( 'C:/Users/84678/Desktop/做市商效率/流动性/流动性.xlsx',sheet_name = "ami_t",index_col = 0)
amihud_c = pd.read_excel( 'C:/Users/84678/Desktop/做市商效率/流动性/流动性.xlsx',sheet_name = "ami_c",index_col = 0)

In [58]:
# 非零交易天数
def non(df):
    df_non = pd.DataFrame()
    for i in range(df.shape[0]):
        non_temp = pd.DataFrame()
        r = df.iloc[i]
        
        for day in [-20,-50,-100,20,50,100]:
            if day>0:
                date1 = r['做市后1交易日']
                date2 = r['做市后'+str(day)+'交易日']
                r1 = r.to_frame().T
                nonzero = r1.loc[:,date1:date2]
            else:
                date1 = r['做市前1交易日']
                date2 = r['做市前'+str(-day)+'交易日']
                r1 = r.to_frame().T
                nonzero = r1.loc[:,date2:date1]
                
            non = nonzero.gt(0).sum(axis=1)
            non_temp[day] = non
            
        df_non = pd.concat([df_non,non_temp],0)
    return df_non

In [59]:
non_t = non(nonzero_t)
non_c = non(nonzero_c)

C:\Users\84678\AppData\Local\Temp\ipykernel_28276\3560751832.py:23: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  df_non = pd.concat([df_non,non_temp],0)
C:\Users\84678\AppData\Local\Temp\ipykernel_28276\3560751832.py:23: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  df_non = pd.concat([df_non,non_temp],0)
C:\Users\84678\AppData\Local\Temp\ipykernel_28276\3560751832.py:23: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  df_non = pd.concat([df_non,non_temp],0)
C:\Users\84678\AppData\Local\Temp\ipykernel_28276\3560751832.py:23: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  df_non = pd.concat([df_non,non_temp],0)
C:\Users\84678\AppData\Local\Temp\ipykernel_28276\3560751832.py:23: Futu

In [60]:
non_t_b = non_t.iloc[:,:3]
non_t_a = non_t.iloc[:,3:]
non_c_b = non_c.iloc[:,:3]
non_c_a = non_c.iloc[:,3:]

In [61]:
non_t_b.columns = [20,50,100]
non_c_b.columns = [20,50,100]

In [62]:
non_t_diff = non_t_a - non_t_b
non_c_diff = non_c_a - non_c_b 

In [63]:
non_t.describe()

,-20,-50,-100,20,50,100
count,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000
mean,6.068000,13.976000,24.138000,6.566000,16.974000,36.250000
std,7.525466,17.593592,32.273986,7.688685,18.306709,35.193589
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
50%,1.000000,3.000000,4.000000,2.000000,8.500000,25.000000
75%,13.000000,28.250000,43.000000,14.000000,33.000000,65.250000
max,20.000000,50.000000,100.000000,20.000000,50.000000,100.000000


In [64]:
non_c.describe()

,-20,-50,-100,20,50,100
count,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000
mean,1.518000,3.814000,7.264000,1.508000,4.022000,8.180000
std,3.711229,8.693363,15.861446,3.650286,8.387265,16.484146
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000
75%,1.000000,3.000000,5.000000,1.000000,3.000000,7.000000
max,20.000000,47.000000,94.000000,20.000000,44.000000,89.000000


In [65]:
for i in [0,1,2]:
    print(ss.ttest_ind(non_t_a.iloc[:,i],non_t_b.iloc[:,i]))
    print(ss.ranksums(non_t_a.iloc[:,i],non_t_b.iloc[:,i]))

Ttest_indResult(statistic=1.0350387925212723, pvalue=0.3009015260691749)
RanksumsResult(statistic=0.9094221150909975, pvalue=0.36312735259469586)
Ttest_indResult(statistic=2.6402671402339344, pvalue=0.008413376928133716)
RanksumsResult(statistic=2.9832680435690353, pvalue=0.0028518811351048758)
Ttest_indResult(statistic=5.671716346598302, pvalue=1.8509155033660107e-08)
RanksumsResult(statistic=6.2274502335697886, pvalue=4.740872600187762e-10)


In [66]:
for i in [0,1,2]:
    print(ss.ttest_ind(non_c_a.iloc[:,i],non_c_b.iloc[:,i]))
    print(ss.ranksums(non_c_a.iloc[:,i],non_c_b.iloc[:,i]))

Ttest_indResult(statistic=-0.04295541553631207, pvalue=0.9657456666397322)
RanksumsResult(statistic=0.1874465038569453, pvalue=0.8513105639008646)
Ttest_indResult(statistic=0.3850257658640946, pvalue=0.7003003651745472)
RanksumsResult(statistic=1.1038759648865202, pvalue=0.26964694631692987)
Ttest_indResult(statistic=0.8953645789742569, pvalue=0.3708081247651438)
RanksumsResult(statistic=1.9158521755191757, pvalue=0.05538391433044761)


In [67]:
for i in [0,1,2]:
    print(ss.ttest_ind(non_t_diff.iloc[:,i],non_c_diff.iloc[:,i]))
    print(ss.ranksums(non_t_diff.iloc[:,i],non_c_diff.iloc[:,i]))

Ttest_indResult(statistic=2.52212691255939, pvalue=0.01181938477619243)
RanksumsResult(statistic=2.213007439226973, pvalue=0.026897132770660512)
Ttest_indResult(statistic=4.925185088578016, pvalue=9.85955392320778e-07)
RanksumsResult(statistic=4.089881252962813, pvalue=4.3159409498248446e-05)
Ttest_indResult(statistic=8.906898935547297, pvalue=2.437539813936746e-18)
RanksumsResult(statistic=8.022206712087923, pvalue=1.0386216412680342e-15)


In [68]:
#turn/vol
def get_data(df):
    df_liq = pd.DataFrame()
    
    for i in range(df.shape[0]):
        liq_temp = pd.DataFrame()
        r = df.iloc[i]
        
        for day in [-20,-50,-100,20,50,100]:
            if day>0:
                date = r['做市后'+str(day)+'交易日']
                r1 = r.to_frame().T
                liq_temp[day] = r1.loc[:,date].astype(float)
            else:
                date = r['做市前'+str(-day)+'交易日']
                r1 = r.to_frame().T
                liq_temp[day] = r1.loc[:,date].astype(float)
            
        df_liq = pd.concat([df_liq,liq_temp],0)
    return df_liq

In [69]:
turn_t = get_data(nonzero_t)
turn_c = get_data(nonzero_c)
vol_t = get_data(volume_t)
vol_c = get_data(volume_c)

C:\Users\84678\AppData\Local\Temp\ipykernel_28276\2759033866.py:19: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  df_liq = pd.concat([df_liq,liq_temp],0)
C:\Users\84678\AppData\Local\Temp\ipykernel_28276\2759033866.py:19: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  df_liq = pd.concat([df_liq,liq_temp],0)
C:\Users\84678\AppData\Local\Temp\ipykernel_28276\2759033866.py:19: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  df_liq = pd.concat([df_liq,liq_temp],0)
C:\Users\84678\AppData\Local\Temp\ipykernel_28276\2759033866.py:19: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  df_liq = pd.concat([df_liq,liq_temp],0)
C:\Users\84678\AppData\Local\Temp\ipykernel_28276\2759033866.py:19: Futu

In [70]:
# turn_t.to_excel("C:/Users/84678/Desktop/做市商效率/流动性数据/turn_t.xlsx" , index = False)
# turn_c.to_excel("C:/Users/84678/Desktop/做市商效率/流动性数据/turn_c.xlsx" , index = False)
# vol_t.to_excel("C:/Users/84678/Desktop/做市商效率/流动性数据/vol_t.xlsx" , index = False)
# vol_c.to_excel("C:/Users/84678/Desktop/做市商效率/流动性数据/vol_c.xlsx" , index = False)

In [71]:
# for i in range(5):
#     turn_t.iloc[:,i] = winsorize(turn_t.iloc[:,i],limits=[0.03,0.03])
#     turn_c.iloc[:,i] = winsorize(turn_c.iloc[:,i],limits=[0.03,0.03])

In [72]:
# for i in range(5):
#     vol_t.iloc[:,i] = winsorize(vol_t.iloc[:,i],limits=[0.03,0.03])
#     vol_c.iloc[:,i] = winsorize(vol_c.iloc[:,i],limits=[0.03,0.03])

In [73]:
turn_t_b = turn_t.iloc[:,:3]
turn_t_a = turn_t.iloc[:,3:]
turn_c_b = turn_c.iloc[:,:3]
turn_c_a = turn_c.iloc[:,3:]
turn_t_b.columns = [20,50,100]
turn_c_b.columns = [20,50,100]

In [74]:
turn_t.describe()

,-20,-50,-100,20,50,100
count,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000
mean,0.097939,0.099651,0.072818,0.125944,0.165160,0.092356
std,0.498964,0.673518,0.426300,1.288583,1.175236,0.593663
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.016491,0.000000,0.000000,0.017644,0.025370,0.024941
max,7.474434,12.752475,7.408737,28.000000,20.000000,11.879687


In [75]:
turn_c.describe()

,-20,-50,-100,20,50,100
count,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000
mean,0.197170,0.046374,0.045743,0.047067,0.311629,0.204475
std,1.812463,0.506842,0.377051,0.380163,4.718856,3.147951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,22.941417,9.243028,5.291854,5.195105,100.000000,69.252078


In [76]:
for i in [0,1,2]:
    print(ss.ttest_ind(turn_t_a.iloc[:,i],turn_t_b.iloc[:,i]))
    print(ss.ranksums(turn_t_a.iloc[:,i],turn_t_b.iloc[:,i]))

Ttest_indResult(statistic=0.45317964411068784, pvalue=0.6505178990317502)
RanksumsResult(statistic=0.5214998235225645, pvalue=0.6020186284838223)
Ttest_indResult(statistic=1.081423884895936, pvalue=0.2797698407216575)
RanksumsResult(statistic=3.4771764424584513, pvalue=0.0005067242360626056)
Ttest_indResult(statistic=0.5977422830724578, pvalue=0.5501475132107295)
RanksumsResult(statistic=5.443612896471732, pvalue=5.221061700832751e-08)


In [77]:
for i in [0,1,2]:
    print(ss.ttest_ind(turn_c_a.iloc[:,i],turn_c_b.iloc[:,i]))
    print(ss.ranksums(turn_c_a.iloc[:,i],turn_c_b.iloc[:,i]))

Ttest_indResult(statistic=-1.8124064022184145, pvalue=0.07022382175770053)
RanksumsResult(statistic=-0.010620508688156362, pvalue=0.99152621939045)
Ttest_indResult(statistic=1.2497437191369805, pvalue=0.21168617330753395)
RanksumsResult(statistic=0.2692353697337783, pvalue=0.7877485636990964)
Ttest_indResult(statistic=1.1195139241950394, pvalue=0.263190313041124)
RanksumsResult(statistic=0.2714251653395838, pvalue=0.786064049110743)


In [78]:
#交易量
vol_t_b = vol_t.iloc[:,:3]
vol_t_a = vol_t.iloc[:,3:]
vol_c_b = vol_c.iloc[:,:3]
vol_c_a = vol_c.iloc[:,3:]
vol_t_b.columns = [20,50,100]
vol_c_b.columns = [20,50,100]

In [79]:
vol_t.describe()

,-20,-50,-100,20,50,100
count,5.000000e+02,5.000000e+02,500.000000,5.000000e+02,5.000000e+02,5.000000e+02
mean,2.194138e+04,2.896677e+04,17859.820000,2.266212e+04,2.679286e+04,2.162771e+04
std,1.116733e+05,2.262620e+05,88191.899621,1.246941e+05,1.523016e+05,8.940821e+04
min,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00
75%,3.000000e+03,0.000000e+00,0.000000,3.000000e+03,4.000000e+03,4.000000e+03
max,1.823000e+06,4.101494e+06,1000000.000000,1.735000e+06,2.389000e+06,1.057500e+06


In [80]:
vol_c.describe()

,-20,-50,-100,20,50,100
count,5.000000e+02,5.000000e+02,5.000000e+02,500.00000,5.000000e+02,5.000000e+02
mean,1.728280e+04,6.563392e+03,6.475632e+03,5448.36200,2.518780e+04,1.837109e+04
std,2.008961e+05,7.132951e+04,6.916227e+04,43010.07988,2.629768e+05,1.997238e+05
min,0.000000e+00,0.000000e+00,0.000000e+00,0.00000,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.00000,0.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00,0.000000e+00,0.00000,0.000000e+00,0.000000e+00
75%,0.000000e+00,0.000000e+00,0.000000e+00,0.00000,0.000000e+00,0.000000e+00
max,4.200000e+06,1.350000e+06,1.280000e+06,600000.00000,4.832000e+06,3.800000e+06


In [81]:
for i in [0,1,2]:
    print(ss.ttest_ind(vol_t_a.iloc[:,i],vol_t_b.iloc[:,i]))
    print(ss.ranksums(vol_t_a.iloc[:,i],vol_t_b.iloc[:,i]))

Ttest_indResult(statistic=0.0962796146035516, pvalue=0.9233178306772414)
RanksumsResult(statistic=0.5127406410993428, pvalue=0.6081327567845884)
Ttest_indResult(statistic=-0.1782255671823894, pvalue=0.8585820013074439)
RanksumsResult(statistic=3.5273227618313956, pvalue=0.0004197847017921766)
Ttest_indResult(statistic=0.6708806553225293, pvalue=0.5024517898550436)
RanksumsResult(statistic=5.5618618591852265, pvalue=2.669115422454513e-08)


In [82]:
for i in [0,1,2]:
    print(ss.ttest_ind(vol_c_a.iloc[:,i],vol_c_b.iloc[:,i]))
    print(ss.ranksums(vol_c_a.iloc[:,i],vol_c_b.iloc[:,i]))

Ttest_indResult(statistic=-1.2880408482543162, pvalue=0.1980303048932152)
RanksumsResult(statistic=-0.005802958355384405, pvalue=0.9953699351069902)
Ttest_indResult(statistic=1.5283921275168517, pvalue=0.12673207472449785)
RanksumsResult(statistic=0.2743813894074211, pvalue=0.7837915435087748)
Ttest_indResult(statistic=1.258471621724041, pvalue=0.20851556429232967)
RanksumsResult(statistic=0.2774471032555487, pvalue=0.7814368174550805)


In [83]:
# 价格冲击指标
def get_ami(df):
    df_ami = pd.DataFrame()
    for i in range(df.shape[0]):
        ami_temp = pd.DataFrame()
        r = df.iloc[i]
        
        for day in [-20,-50,-100,20,50,100]:
            if day>0:
                date1 = r['做市后1交易日']
                date2 = r['做市后'+str(day)+'交易日']
                r1 = r.to_frame().T
                ami = r1.loc[:,date1:date2].mean(1)
            else:
                date1 = r['做市前1交易日']
                date2 = r['做市前'+str(-day)+'交易日']
                r1 = r.to_frame().T
                ami = r1.loc[:,date2:date1].mean(1)
            ami_temp[day] = ami
            
        df_ami = pd.concat([df_ami,ami_temp],0)
    return df_ami

In [84]:
ami_t1 = get_ami(amihud_t)
ami_c1 = get_ami(amihud_c)

C:\Users\84678\AppData\Local\Temp\ipykernel_28276\3536703686.py:21: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  df_ami = pd.concat([df_ami,ami_temp],0)
C:\Users\84678\AppData\Local\Temp\ipykernel_28276\3536703686.py:21: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  df_ami = pd.concat([df_ami,ami_temp],0)
C:\Users\84678\AppData\Local\Temp\ipykernel_28276\3536703686.py:21: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  df_ami = pd.concat([df_ami,ami_temp],0)
C:\Users\84678\AppData\Local\Temp\ipykernel_28276\3536703686.py:21: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  df_ami = pd.concat([df_ami,ami_temp],0)
C:\Users\84678\AppData\Local\Temp\ipykernel_28276\3536703686.py:21: Futu

In [85]:
# for i in range(5):
#     print('偏度:',ss.skew(ami_t1.iloc[:,i]))
#     print('峰值:',ss.kurtosis(ami_t1.iloc[:,i]))

In [86]:
# for i in range(5):
#     print('偏度:',ss.skew(ami_t.iloc[:,i]))
#     print('峰值:',ss.kurtosis(ami_t.iloc[:,i]))

In [87]:
def zero_out(df):
    for i in range(df.shape[0]):
        for j in range(df.shape[1]):
            if df.iloc[i,j] == 0:
                df.iloc[i,j] = 1
    return df

In [88]:
ami_t = zero_out(ami_t1)
ami_c = zero_out(ami_c1)
ami_t = np.log(ami_t)
ami_c = np.log(ami_c)

In [89]:
ami_t_b = ami_t.iloc[:,:3]
ami_t_a = ami_t.iloc[:,3:]
ami_c_b = ami_c.iloc[:,:3]
ami_c_a = ami_c.iloc[:,3:]
ami_t_b.columns = [20,50,100]
ami_c_b.columns = [20,50,100]

In [90]:
ami_t.describe()

,-20,-50,-100,20,50,100
count,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000
mean,-5.462784,-5.845168,-6.093380,-5.899536,-6.913400,-8.027277
std,5.449856,5.405847,5.419113,5.564268,5.217650,4.659742
min,-17.272385,-18.188676,-18.881823,-20.229573,-18.652393,-19.345541
25%,-10.309156,-10.582288,-10.535450,-10.638549,-10.715200,-10.889700
50%,-6.979288,-8.246610,-8.517445,-8.424216,-9.389793,-9.684281
75%,0.000000,0.000000,0.000000,0.000000,0.000000,-7.630218
max,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [91]:
ami_c.describe()

,-20,-50,-100,20,50,100
count,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000
mean,-2.381189,-3.578657,-4.525668,-2.605384,-3.993231,-5.442216
std,4.451208,5.178609,5.594619,4.674431,5.405251,5.774992
min,-17.206658,-22.268096,-22.961243,-18.419470,-19.669121,-20.234066
25%,0.000000,-8.294127,-9.473322,-5.534170,-8.842119,-9.946129
50%,0.000000,0.000000,0.000000,0.000000,0.000000,-4.465902
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [92]:
for i in [0,1,2]:
    print(ss.ttest_ind(ami_t_a.iloc[:,i],ami_t_b.iloc[:,i]))
    print(ss.ranksums(ami_t_a.iloc[:,i],ami_t_b.iloc[:,i]))

Ttest_indResult(statistic=-1.2538977879961644, pvalue=0.2101727876799884)
RanksumsResult(statistic=-1.1640953440461697, pvalue=0.24438537528684)
Ttest_indResult(statistic=-3.179289667769065, pvalue=0.0015218216640737624)
RanksumsResult(statistic=-2.730565630659088, pvalue=0.00632257427466793)
Ttest_indResult(statistic=-6.050523907591561, pvalue=2.039669130912228e-09)
RanksumsResult(statistic=-4.999741327174971, pvalue=5.740727892322455e-07)


In [93]:
for i in [0,1,2]:
    print(ss.ttest_ind(ami_c_a.iloc[:,i],ami_c_b.iloc[:,i]))
    print(ss.ranksums(ami_c_a.iloc[:,i],ami_c_b.iloc[:,i]))

Ttest_indResult(statistic=-0.7766638320851793, pvalue=0.4375411646960884)
RanksumsResult(statistic=-0.5310254344078181, pvalue=0.5954011540593078)
Ttest_indResult(statistic=-1.2383929884413216, pvalue=0.21586159270895727)
RanksumsResult(statistic=-1.0944598437815567, pvalue=0.2737533499086122)
Ttest_indResult(statistic=-2.548911177527521, pvalue=0.01095470537634523)
RanksumsResult(statistic=-2.394979454069405, pvalue=0.016621296626789083)


## 倍差法模型

$Liq = a_0 + a_1Timedum_i + a_2Makerdum_i + a_3Timedum_i × Makerdum_i + a_4Lnprice_i + a_5Lnvolatility_i + a_6Lnoutstanding_i + ε_i$

liq取non,vol/1000,turn和ami

In [94]:
treatment = pd.read_excel( 'C:/Users/84678/Desktop/做市商效率/流动性/两组数据对比.xlsx',sheet_name = "treatment",index_col = 0)
control = pd.read_excel( 'C:/Users/84678/Desktop/做市商效率/流动性/两组数据对比.xlsx',sheet_name = "control",index_col = 0)

In [95]:
did_t1 = pd.DataFrame()
did_t1['lnprice'] = treatment['meanprice']
did_t1['lnvolatility'] = treatment['volatility']
did_t1['lnoutstanding'] = treatment['outstanding']
did_t = zero_out(did_t1)
did_t = np.log(did_t)

In [96]:
did_c1 = pd.DataFrame()
did_c1['lnprice'] = control['meanprice']
did_c1['lnvolatility'] = control['volatility']
did_c1['lnoutstanding'] = control['outstanding']
did_c = zero_out(did_c1)
did_c = np.log(did_c)

In [97]:
did_t['makerdum'] = 1
did_c['makerdum'] = 0

In [98]:
did = pd.concat([did_t,did_c],0)
did

C:\Users\84678\AppData\Local\Temp\ipykernel_28276\224637530.py:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  did = pd.concat([did_t,did_c],0)


,lnprice,lnvolatility,lnoutstanding,makerdum
430014.NQ,1.275698,3.292584,17.962106,1
430022.NQ,1.099379,1.531923,16.843758,1
430027.NQ,2.143402,2.235992,16.871867,1
430033.NQ,0.611286,3.922174,16.979470,1
430037.NQ,1.187996,3.189484,18.163833,1
...,...,...,...,...
834183.NQ,2.491774,3.242200,0.000000,0
834523.NQ,1.792395,-1.570523,13.830399,0
833841.NQ,2.473576,2.982810,16.502923,0
835085.NQ,0.000000,0.000000,0.000000,0


In [99]:
df_t_a = pd.concat([non_t_a,turn_t_a,vol_t_a/1000,ami_t_a],1)
df_t_b = pd.concat([non_t_b,turn_t_b,vol_t_b/1000,ami_t_b],1)
df_c_a = pd.concat([non_c_a,turn_c_a,vol_c_a/1000,ami_c_a],1)
df_c_b = pd.concat([non_c_b,turn_c_b,vol_c_b/1000,ami_c_b],1)

C:\Users\84678\AppData\Local\Temp\ipykernel_28276\3425739293.py:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  df_t_a = pd.concat([non_t_a,turn_t_a,vol_t_a/1000,ami_t_a],1)
C:\Users\84678\AppData\Local\Temp\ipykernel_28276\3425739293.py:2: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  df_t_b = pd.concat([non_t_b,turn_t_b,vol_t_b/1000,ami_t_b],1)
C:\Users\84678\AppData\Local\Temp\ipykernel_28276\3425739293.py:3: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  df_c_a = pd.concat([non_c_a,turn_c_a,vol_c_a/1000,ami_c_a],1)
C:\Users\84678\AppData\Local\Temp\ipykernel_28276\3425739293.py:4: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  df_c_b = pd.concat([non_c_b,turn_c_b,vol_c_b/1000,

In [100]:
df_a = pd.concat([df_t_a,df_c_a],0)
df_a['timedum'] = 1
df_b = pd.concat([df_t_b,df_c_b],0)
df_b['timedum'] = 0

C:\Users\84678\AppData\Local\Temp\ipykernel_28276\1888467548.py:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  df_a = pd.concat([df_t_a,df_c_a],0)
C:\Users\84678\AppData\Local\Temp\ipykernel_28276\1888467548.py:3: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  df_b = pd.concat([df_t_b,df_c_b],0)


In [101]:
did.to_excel("C:/Users/84678/Desktop/做市商效率/流动性/倍差法模型/did.xlsx" , index = True)
df_a.to_excel("C:/Users/84678/Desktop/做市商效率/流动性/倍差法模型/df_a.xlsx" , index = True)
df_b.to_excel("C:/Users/84678/Desktop/做市商效率/流动性/倍差法模型/df_b.xlsx" , index = True)

In [102]:
df_did = pd.read_excel( 'C:/Users/84678/Desktop/做市商效率/流动性/倍差法模型/df_did.xlsx',index_col = 0)

In [103]:
for i in range(15):
    df_did.iloc[:,i] = winsorize(df_did.iloc[:,i],limits=[0.03,0.03])

In [104]:
# for i in range(4,16):
#     print('偏度:',ss.skew(df_did.iloc[:,i]))
#     print('峰值:',ss.kurtosis(df_did.iloc[:,i]))

In [117]:
#对20天数据进行回归
reg_did = smf.ols(formula='non_20~timedum+makerdum+timedum*makerdum+lnprice+lnvolatility+lnoutstanding',data=df_did)
results_did = reg_did.fit()
results_did.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 non_20   R-squared:                       0.194
Model:                            OLS   Adj. R-squared:                  0.192
Method:                 Least Squares   F-statistic:                     79.94
Date:                Sat, 22 Jul 2023   Prob (F-statistic):           8.94e-90
Time:                        09:26:26   Log-Likelihood:                -6346.5
No. Observations:                2000   AIC:                         1.271e+04
Df Residuals:                    1993   BIC:                         1.275e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           -1.2941      0.514     -2.517      0.012      -2.303      -0.286
timedum             -0.0100      0.366     -0.027      0.978      -0.728       0.708
makerdum             4.0363      0.370     10.911      0.000       3.311       4.762
timedum:makerdum     0.5080      0.518      0.981      0.327      -0.508       1.524
lnprice              0.5286      0.144      3.667      0.000       0.246       0.811
lnvolatility         0.6529      0.076      8.555      0.000       0.503       0.803
lnoutstanding        0.0783      0.027      2.912      0.004       0.026       0.131
==============================================================================
Omnibus:                      274.998   Durbin-Watson:                   1.725
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              395.562
Skew:                           1.063   Prob(JB):                     1.27e-86
Kurtosis:                       3.480   Cond. No.                         83.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [106]:
# reg_did = smf.ols(formula='turn_20~timedum+makerdum+timedum*makerdum+lnprice+lnvolatility+lnoutstanding',data=df_did)
# results_did = reg_did.fit()
# results_did.summary()

In [107]:
# reg_did = smf.ols(formula='vol_20~timedum+makerdum+timedum*makerdum+lnprice+lnvolatility+lnoutstanding',data=df_did)
# results_did = reg_did.fit()
# results_did.summary()

In [108]:
reg_did = smf.ols(formula='ami_20~timedum+makerdum+timedum*makerdum+lnprice+lnvolatility+lnoutstanding',data=df_did)
results_did = reg_did.fit()
results_did.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 ami_20   R-squared:                       0.165
Model:                            OLS   Adj. R-squared:                  0.163
Method:                 Least Squares   F-statistic:                     65.65
Date:                Fri, 21 Jul 2023   Prob (F-statistic):           1.22e-74
Time:                        00:46:30   Log-Likelihood:                -5956.0
No. Observations:                2000   AIC:                         1.193e+04
Df Residuals:                    1993   BIC:                         1.197e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           -0.1641      0.423     -0.388      0.698      -0.994       0.665
timedum             -0.2093      0.301     -0.695      0.487      -0.800       0.381
makerdum            -2.6009      0.304     -8.547      0.000      -3.198      -2.004
timedum:makerdum    -0.1917      0.426     -0.450      0.653      -1.027       0.644
lnprice             -0.5614      0.119     -4.734      0.000      -0.794      -0.329
lnvolatility        -0.6067      0.063     -9.664      0.000      -0.730      -0.484
lnoutstanding       -0.0358      0.022     -1.616      0.106      -0.079       0.008
==============================================================================
Omnibus:                      201.292   Durbin-Watson:                   1.817
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              125.564
Skew:                          -0.487   Prob(JB):                     5.42e-28
Kurtosis:                       2.253   Cond. No.                         83.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [109]:
reg_did = smf.ols(formula='non_50~timedum+makerdum+timedum*makerdum+lnprice+lnvolatility+lnoutstanding',data=df_did)
results_did = reg_did.fit()
results_did.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 non_50   R-squared:                       0.206
Model:                            OLS   Adj. R-squared:                  0.204
Method:                 Least Squares   F-statistic:                     86.35
Date:                Fri, 21 Jul 2023   Prob (F-statistic):           2.11e-96
Time:                        00:46:34   Log-Likelihood:                -8043.9
No. Observations:                2000   AIC:                         1.610e+04
Df Residuals:                    1993   BIC:                         1.614e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           -2.6789      1.201     -2.230      0.026      -5.035      -0.323
timedum              0.2080      0.856      0.243      0.808      -1.470       1.886
makerdum             8.8807      0.864     10.274      0.000       7.185      10.576
timedum:makerdum     2.7740      1.210      2.293      0.022       0.401       5.147
lnprice              1.3942      0.337      4.139      0.000       0.734       2.055
lnvolatility         1.5743      0.178      8.828      0.000       1.225       1.924
lnoutstanding        0.1561      0.063      2.484      0.013       0.033       0.279
==============================================================================
Omnibus:                      273.850   Durbin-Watson:                   1.718
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              393.106
Skew:                           1.046   Prob(JB):                     4.35e-86
Kurtosis:                       3.584   Cond. No.                         83.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [110]:
# reg_did = smf.ols(formula='turn_50~timedum+makerdum+timedum*makerdum+lnprice+lnvolatility+lnoutstanding',data=df_did)
# results_did = reg_did.fit()
# results_did.summary()

In [111]:
# reg_did = smf.ols(formula='vol_50~timedum+makerdum+timedum*makerdum+lnprice+lnvolatility+lnoutstanding',data=df_did)
# results_did = reg_did.fit()
# results_did.summary()

In [112]:
reg_did = smf.ols(formula='ami_50~timedum+makerdum+timedum*makerdum+lnprice+lnvolatility+lnoutstanding',data=df_did)
results_did = reg_did.fit()
results_did.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 ami_50   R-squared:                       0.151
Model:                            OLS   Adj. R-squared:                  0.148
Method:                 Least Squares   F-statistic:                     58.89
Date:                Fri, 21 Jul 2023   Prob (F-statistic):           2.64e-67
Time:                        00:46:37   Log-Likelihood:                -6034.1
No. Observations:                2000   AIC:                         1.208e+04
Df Residuals:                    1993   BIC:                         1.212e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           -0.7880      0.440     -1.792      0.073      -1.651       0.075
timedum             -0.3762      0.313     -1.201      0.230      -0.990       0.238
makerdum            -1.8154      0.316     -5.737      0.000      -2.436      -1.195
timedum:makerdum    -0.6787      0.443     -1.532      0.126      -1.547       0.190
lnprice             -0.8773      0.123     -7.115      0.000      -1.119      -0.635
lnvolatility        -0.5812      0.065     -8.902      0.000      -0.709      -0.453
lnoutstanding       -0.0426      0.023     -1.850      0.064      -0.088       0.003
==============================================================================
Omnibus:                      349.564   Durbin-Watson:                   1.859
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               98.397
Skew:                          -0.273   Prob(JB):                     4.30e-22
Kurtosis:                       2.060   Cond. No.                         83.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [113]:
reg_did = smf.ols(formula='non_100~timedum+makerdum+timedum*makerdum+lnprice+lnvolatility+lnoutstanding',data=df_did)
results_did = reg_did.fit()
results_did.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                non_100   R-squared:                       0.232
Model:                            OLS   Adj. R-squared:                  0.229
Method:                 Least Squares   F-statistic:                     100.2
Date:                Fri, 21 Jul 2023   Prob (F-statistic):          2.16e-110
Time:                        00:46:41   Log-Likelihood:                -9291.5
No. Observations:                2000   AIC:                         1.860e+04
Df Residuals:                    1993   BIC:                         1.864e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           -5.1914      2.242     -2.316      0.021      -9.588      -0.795
timedum              0.9160      1.597      0.574      0.566      -2.215       4.047
makerdum            14.3069      1.613      8.870      0.000      11.144      17.470
timedum:makerdum    11.1060      2.258      4.919      0.000       6.678      15.534
lnprice              2.8246      0.629      4.494      0.000       1.592       4.057
lnvolatility         3.0260      0.333      9.093      0.000       2.373       3.679
lnoutstanding        0.2828      0.117      2.412      0.016       0.053       0.513
==============================================================================
Omnibus:                      281.830   Durbin-Watson:                   1.749
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              412.365
Skew:                           1.036   Prob(JB):                     2.86e-90
Kurtosis:                       3.810   Cond. No.                         83.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [114]:
# reg_did = smf.ols(formula='turn_100~timedum+makerdum+timedum*makerdum+lnprice+lnvolatility+lnoutstanding',data=df_did)
# results_did = reg_did.fit()
# results_did.summary()

In [ ]:
# reg_did = smf.ols(formula='vol_100~timedum+makerdum+timedum*makerdum+lnprice+lnvolatility+lnoutstanding',data=df_did)
# results_did = reg_did.fit()
# results_did.summary()

In [ ]:
reg_did = smf.ols(formula='ami_100~timedum+makerdum+timedum*makerdum+lnprice+lnvolatility+lnoutstanding',data=df_did)
results_did = reg_did.fit()
results_did.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                ami_100   R-squared:                       0.141
Model:                            OLS   Adj. R-squared:                  0.139
Method:                 Least Squares   F-statistic:                     54.65
Date:                Fri, 21 Jul 2023   Prob (F-statistic):           1.21e-62
Time:                        00:46:46   Log-Likelihood:                -6064.9
No. Observations:                2000   AIC:                         1.214e+04
Df Residuals:                    1993   BIC:                         1.218e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           -1.3056      0.447     -2.923      0.004      -2.182      -0.430
timedum             -0.8940      0.318     -2.811      0.005      -1.518      -0.270
makerdum            -1.1669      0.321     -3.631      0.000      -1.797      -0.537
timedum:makerdum    -1.0534      0.450     -2.342      0.019      -1.936      -0.171
lnprice             -0.9341      0.125     -7.460      0.000      -1.180      -0.689
lnvolatility        -0.5318      0.066     -8.022      0.000      -0.662      -0.402
lnoutstanding       -0.0689      0.023     -2.950      0.003      -0.115      -0.023
==============================================================================
Omnibus:                      343.511   Durbin-Watson:                   1.929
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               82.487
Skew:                          -0.154   Prob(JB):                     1.23e-18
Kurtosis:                       2.054   Cond. No.                         83.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## 定价效率

$pricing = a_0 + a_1Timedum_i + a_2Makerdum_i + a_3Timedum_i × Makerdum_i + a_4Lnprice_i + a_5Lnvolatility_i + a_6Lnoutstanding_i + ε_i$

$pricing = a_0 + a_1number + a_2Lnprice_i + a_3Lnvolatility_i + a_4Lnoutstanding_i + ε_i$

In [540]:
# week = pd.read_excel( 'C:/Users/84678/Desktop/做市商效率/定价效率/df_effi.xlsx',sheet_name = "week")
# date = pd.read_excel( 'C:/Users/84678/Desktop/做市商效率/定价效率/df_effi.xlsx',sheet_name = "date")

In [541]:
# for i in range(date.shape[0]):
#     for j in range(week.shape[0]):
#         if date.iloc[i,0]>week.iloc[j,0]:
#             continue
#         else: 
#             date.iloc[i,1] = week.iloc[j,0]
#             break

In [542]:
# date.to_excel("C:/Users/84678/Desktop/做市商效率/定价效率/date.xlsx" , index = True)

In [543]:
day_t = pd.read_excel( 'C:/Users/84678/Desktop/做市商效率/定价效率/df_effi.xlsx',sheet_name = "day_t",index_col = 0)
day_c = pd.read_excel( 'C:/Users/84678/Desktop/做市商效率/定价效率/df_effi.xlsx',sheet_name = "day_c",index_col = 0)
week_t = pd.read_excel( 'C:/Users/84678/Desktop/做市商效率/定价效率/df_effi.xlsx',sheet_name = "week_t",index_col = 0)
week_c = pd.read_excel( 'C:/Users/84678/Desktop/做市商效率/定价效率/df_effi.xlsx',sheet_name = "week_c",index_col = 0)
day_market = pd.read_excel( 'C:/Users/84678/Desktop/做市商效率/定价效率/df_effi.xlsx',sheet_name = "day_market",index_col = 0)
week_market = pd.read_excel( 'C:/Users/84678/Desktop/做市商效率/定价效率/df_effi.xlsx',sheet_name = "week_market",index_col = 0)

In [549]:
# 获取前100天的收益率数据
day_ret = day_market.iloc[0].to_frame().T
def get_r2(df):
    df_r2 = pd.DataFrame()
    for i in range(df.shape[0]):
        r2_temp = pd.DataFrame()
        r = df.iloc[i]
        date_b1 = r['做市前1交易日']
        date_b2 = r['做市前100交易日']
        date_a1 = r['做市后1交易日']
        date_a2 = r['做市后100交易日']
        
        r1 = r.to_frame().T
        ret_b = r1.loc[:,date_b2:date_b1].T
        ret_a = r1.loc[:,date_a1:date_a2].T
        market_b = day_ret.loc[:,date_b2:date_b1].T
        market_a = day_ret.loc[:,date_a1:date_a2].T
        reg_b = sm.OLS(ret_b.astype(float),market_b.astype(float)).fit()
        reg_a = sm.OLS(ret_a.astype(float),market_a.astype(float)).fit()
        
        r2_temp = pd.DataFrame([reg_b.rsquared,reg_a.rsquared]).T
        r2_temp.columns = ['b','a']
        
        r2_temp['做市首日'] = r['做市首日']
        r2_temp['做市前100交易日'] = date_b2
        r2_temp['做市前1交易日'] = date_b1
        r2_temp['做市后1交易日'] = date_a1
        r2_temp['做市后100交易日'] = date_a2
        
        df_r2 = pd.concat([df_r2,r2_temp],0)
    
    df_r2.index = df.index.values
    df_r2 = df_r2.dropna(how='any')
    return df_r2

In [550]:
r2_t = get_r2(day_t)
r2_c = get_r2(day_c)

C:\Users\84678\AppData\Local\Temp\ipykernel_15880\2294571246.py:30: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  df_r2 = pd.concat([df_r2,r2_temp],0)
C:\Users\84678\AppData\Local\Temp\ipykernel_15880\2294571246.py:30: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  df_r2 = pd.concat([df_r2,r2_temp],0)
C:\Users\84678\AppData\Local\Temp\ipykernel_15880\2294571246.py:30: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  df_r2 = pd.concat([df_r2,r2_temp],0)
C:\Users\84678\AppData\Local\Temp\ipykernel_15880\2294571246.py:30: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  df_r2 = pd.concat([df_r2,r2_temp],0)
C:\Users\84678\AppData\Local\Temp\ipykernel_15880\2294571246.py:30: FutureWarning: I

In [551]:
# 构建syn
syn_t = r2_t
syn_c = r2_c
for i in ['a','b']:
    syn_t[i] = np.log(syn_t[i]/(1-syn_t[i]))
    syn_c[i] = np.log(syn_c[i]/(1-syn_c[i]))

In [552]:
syn_c

,b,a,做市首日,做市前100交易日,做市前1交易日,做市后1交易日,做市后100交易日
430258.NQ,-4.362387,-5.995027,2015/12/8,2015/7/10,2015/12/7,2015/12/9,2016/5/6
831654.NQ,-3.769186,-4.179937,2016/5/30,2015/12/30,2016/5/27,2016/5/31,2016/10/28
430168.NQ,-4.677311,-6.870433,2015/6/17,2015/1/19,2015/6/16,2015/6/18,2015/11/16
430557.NQ,-7.920717,-3.557798,2015/9/25,2015/5/5,2015/9/24,2015/9/28,2016/2/29
430264.NQ,-5.651826,-3.504612,2015/7/8,2015/2/6,2015/7/7,2015/7/9,2015/12/4
...,...,...,...,...,...,...,...
832302.NQ,-4.526318,-5.865405,2016/8/12,2016/3/21,2016/8/11,2016/8/15,2017/1/11
834183.NQ,-7.036989,-3.447750,2016/11/24,2016/6/28,2016/11/23,2016/11/25,2017/4/25
834183.NQ,-7.036989,-3.447750,2016/11/24,2016/6/28,2016/11/23,2016/11/25,2017/4/25
834523.NQ,-6.669010,-6.502349,2016/10/10,2016/5/10,2016/9/30,2016/10/11,2017/3/7


In [553]:
for i in range(2):
    print('偏度:',ss.skew(syn_t.iloc[:,i]))
    print('峰值:',ss.kurtosis(syn_t.iloc[:,i]))

偏度: -0.9759823234588919
峰值: 1.3573787024282256
偏度: -1.0556043052747175
峰值: 2.3057350954514515


In [554]:
syn_t.iloc[:,:2].describe()

,b,a
count,274.000000,274.000000
mean,-6.348090,-5.369808
std,2.646625,2.280207
min,-15.620942,-16.240479
25%,-7.757333,-6.379277
50%,-5.858596,-5.159143
75%,-4.470462,-3.713552
max,-1.068463,-0.479952


In [555]:
syn_c.iloc[:,:2].describe()

,b,a
count,173.000000,173.000000
mean,-6.237225,-6.144731
std,2.360544,2.337171
min,-15.601895,-18.072505
25%,-7.239158,-7.004381
50%,-5.926215,-5.779917
75%,-4.677311,-4.573350
max,-2.125040,-1.825618


In [556]:
print(ss.ttest_ind(syn_t['a'],syn_t['b']))
print(ss.ranksums(syn_t['a'],syn_t['b']))

Ttest_indResult(statistic=4.635417858635266, pvalue=4.460894821115658e-06)
RanksumsResult(statistic=4.558890889266559, pvalue=5.14244781893977e-06)


In [557]:
print(ss.ttest_ind(syn_c['a'],syn_c['b']))
print(ss.ranksums(syn_c['a'],syn_c['b']))

Ttest_indResult(statistic=0.3662317312444052, pvalue=0.714417298504578)
RanksumsResult(statistic=0.5627253351678986, pvalue=0.573621929648634)


In [558]:
mar_log = week_market.iloc[1]

In [559]:
# 计算ρ
def get_corr(df):
    df_corr = pd.DataFrame()
    for i in range(df.shape[0]):
        corr_temp = pd.DataFrame()
        r = df.iloc[i]
        date_b1 = r['做市前1周']
        date_b2 = r['做市前14周']
        date_a1 = r['做市后1周']
        date_a2 = r['做市后14周']
        
        r1 = r
        ret_b = r1.loc[date_b2:date_b1]
        ret_a = r1.loc[date_a1:date_a2]
    
        mar_b = mar_log.loc[date_b2:date_b1]
        mar_a = mar_log.loc[date_a1:date_a2]
        cor_b = abs(ret_b.astype(float).corr(mar_b.astype(float)))
        cor_a = abs(ret_a.astype(float).corr(mar_a.astype(float)))
        
        corr_temp = pd.DataFrame([cor_b,cor_a]).T
        corr_temp.columns = ['b','a']
        
        corr_temp['做市首日'] = r['做市首日']
        corr_temp['做市前14周'] = date_b2
        corr_temp['做市前1周'] = date_b1
        corr_temp['做市后1周'] = date_a1
        corr_temp['做市后14周'] = date_a2
        
        df_corr = pd.concat([df_corr,corr_temp],0)
        
    df_corr.index = df.index.values
    df_corr = df_corr.dropna(how='any')
    return df_corr

In [562]:
corr_t = get_corr(week_t)
corr_c = get_corr(week_c)

C:\Users\84678\AppData\Local\Temp\ipykernel_15880\4096050022.py:30: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  df_corr = pd.concat([df_corr,corr_temp],0)
C:\Users\84678\AppData\Local\Temp\ipykernel_15880\4096050022.py:30: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  df_corr = pd.concat([df_corr,corr_temp],0)
C:\Users\84678\AppData\Local\Temp\ipykernel_15880\4096050022.py:30: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  df_corr = pd.concat([df_corr,corr_temp],0)
C:\Users\84678\AppData\Local\Temp\ipykernel_15880\4096050022.py:30: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  df_corr = pd.concat([df_corr,corr_temp],0)
C:\Users\84678\AppData\Local\Temp\ipykernel_15880\4096050022

In [563]:
#构建copast
# corp_t = np.log(corr_t/(1-corr_t))
# corp_c = np.log(corr_c/(1-corr_c))
corp_t = corr_t
corp_c = corr_c
for i in ['a','b']:
    corp_t[i] = np.log(corp_t[i]/(1-corp_t[i]))
    corp_c[i] = np.log(corp_c[i]/(1-corp_c[i]))
# corp_t = np.log(corr_t)
# corp_c = np.log(corr_c)
# corp_t = corr_t**(1/2)
# corp_c = corr_c**(1/2)

In [564]:
for i in range(2):
    print('偏度:',ss.skew(corp_t.iloc[:,i]))
    print('峰值:',ss.kurtosis(corp_t.iloc[:,i]))

偏度: -0.7977492636104077
峰值: 1.3988419992017453
偏度: -0.7347692017200944
峰值: 1.5533242006920869


In [565]:
corp_t.iloc[:,:2].describe()

,b,a
count,260.000000,260.000000
mean,-1.602471,-1.507196
std,1.331655,1.304641
min,-6.800104,-6.797724
25%,-2.323763,-2.113183
50%,-1.428665,-1.323675
75%,-0.739922,-0.727657
max,1.725967,1.884634


In [566]:
corp_c.iloc[:,:2].describe()

,b,a
count,142.000000,142.000000
mean,-1.719608,-1.583416
std,1.508761,1.271905
min,-9.438050,-6.398483
25%,-2.403655,-2.311347
50%,-1.563894,-1.342488
75%,-0.740460,-0.618150
max,2.300554,1.109741


In [567]:
print(ss.ttest_ind(corp_t['a'],corp_t['b']))
print(ss.ranksums(corp_t['a'],corp_t['b']))

Ttest_indResult(statistic=0.8240697019129399, pvalue=0.4102792212526083)
RanksumsResult(statistic=0.8189475432830777, pvalue=0.41281634288976976)


In [568]:
print(ss.ttest_ind(corp_c['a'],corp_c['b']))
print(ss.ranksums(corp_c['a'],corp_c['b']))

Ttest_indResult(statistic=0.8224175183201111, pvalue=0.41153415983068553)
RanksumsResult(statistic=0.7073470469657506, pvalue=0.4793508353678344)


In [569]:
corp_t['makerdum'] = 1
corp_c['makerdum'] = 0
syn_t['makerdum'] = 1
syn_c['makerdum'] = 0

In [570]:
syn = pd.concat([syn_t,syn_c],0)
corp = pd.concat([corp_t,corp_c],0)

C:\Users\84678\AppData\Local\Temp\ipykernel_15880\2015151442.py:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  syn = pd.concat([syn_t,syn_c],0)
C:\Users\84678\AppData\Local\Temp\ipykernel_15880\2015151442.py:2: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  corp = pd.concat([corp_t,corp_c],0)


In [571]:
syn.to_excel("C:/Users/84678/Desktop/做市商效率/定价效率/syn.xlsx" , index = True)
corp.to_excel("C:/Users/84678/Desktop/做市商效率/定价效率/corp.xlsx" , index = True)

In [572]:
#计算价格滞后指标
md_log1 = day_market.iloc[1].to_frame().T
md_log2 = day_market.iloc[2].to_frame().T
md_log3 = day_market.iloc[3].to_frame().T
md_log4 = day_market.iloc[4].to_frame().T
marday = day_market.iloc[0].to_frame().T


def get_D(df):
    df_D1 = pd.DataFrame()
    df_D2 = pd.DataFrame()
    for i in range(df.shape[0]):
        D1_temp = pd.DataFrame()
        D2_temp = pd.DataFrame()
        r = df.iloc[i]
        date_b1 = r['做市前1交易日']
        date_b2 = r['做市前100交易日']
        date_a1 = r['做市后1交易日']
        date_a2 = r['做市后100交易日']
        
        r1 = r.to_frame().T
        ret_b = r1.loc[:,date_b2:date_b1].T
        ret_a = r1.loc[:,date_a1:date_a2].T
        market_b = marday.loc[:,date_b2:date_b1].T
        market_a = marday.loc[:,date_a1:date_a2].T
        marday_log1_b = md_log1.loc[:,date_b2:date_b1].T 
        marday_log2_b = md_log2.loc[:,date_b2:date_b1].T
        marday_log3_b = md_log3.loc[:,date_b2:date_b1].T
        marday_log4_b = md_log4.loc[:,date_b2:date_b1].T

        marday_log1_a = md_log1.loc[:,date_a1:date_a2].T 
        marday_log2_a = md_log2.loc[:,date_a1:date_a2].T
        marday_log3_a = md_log3.loc[:,date_a1:date_a2].T
        marday_log4_a = md_log4.loc[:,date_a1:date_a2].T
        X_a = np.column_stack((market_a,marday_log1_a,marday_log2_a,marday_log3_a,marday_log4_a))
        X_b = np.column_stack((market_b,marday_log1_b,marday_log2_b,marday_log3_b,marday_log4_b))

        reg_b = sm.OLS(ret_b.astype(float),market_b.astype(float)).fit()
        reg_a = sm.OLS(ret_a.astype(float),market_a.astype(float)).fit()
        reg_b1 = sm.OLS(ret_b.astype(float),X_b).fit()
        reg_a1 = sm.OLS(ret_a.astype(float),X_a).fit()

        D1_b = 1 - reg_b.rsquared/reg_b1.rsquared
        D1_a = 1 - reg_a.rsquared/reg_a1.rsquared

        D1_temp = pd.DataFrame([D1_b,D1_a]).T
        D1_temp.columns = ['b1','a1']
        D1_temp['做市首日'] = r['做市首日']
        D1_temp['做市前100交易日'] = date_b2
        D1_temp['做市前1交易日'] = date_b1
        D1_temp['做市后1交易日'] = date_a1
        D1_temp['做市后100交易日'] = date_a2

        coef_b = abs(reg_b1.params.to_frame().T)
        coef_a = abs(reg_a1.params.to_frame().T)
        D2_b=(coef_b['x2']+coef_b['x3']+coef_b['x4']+coef_b['x5'])/(coef_b['x1']+coef_b['x2']+coef_b['x3']+coef_b['x4']+coef_b['x5'])
        D2_a=(coef_a['x2']+coef_a['x3']+coef_a['x4']+coef_a['x5'])/(coef_a['x1']+coef_a['x2']+coef_a['x3']+coef_a['x4']+coef_a['x5'])

        D2_temp = pd.DataFrame([D2_b,D2_a]).T
        D2_temp.columns = ['b2','a2']
        D2_temp['做市首日'] = r['做市首日']
        D2_temp['做市前100交易日'] = date_b2
        D2_temp['做市前1交易日'] = date_b1
        D2_temp['做市后1交易日'] = date_a1
        D2_temp['做市后100交易日'] = date_a2
        
        df_D1 = pd.concat([df_D1,D1_temp],0)
        df_D2 = pd.concat([df_D2,D2_temp],0)
        df_D = pd.concat([df_D1,df_D2],1)
    
    df_D.index = df.index.values
    # df_D = df_D.dropna(how='any')
    return df_D

In [573]:
D_t = get_D(day_t)
D_c = get_D(day_c)

C:\Users\84678\AppData\Local\Temp\ipykernel_15880\1036127240.py:67: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  df_D1 = pd.concat([df_D1,D1_temp],0)
C:\Users\84678\AppData\Local\Temp\ipykernel_15880\1036127240.py:68: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  df_D2 = pd.concat([df_D2,D2_temp],0)
C:\Users\84678\AppData\Local\Temp\ipykernel_15880\1036127240.py:69: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  df_D = pd.concat([df_D1,df_D2],1)
C:\Users\84678\AppData\Local\Temp\ipykernel_15880\1036127240.py:67: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  df_D1 = pd.concat([df_D1,D1_temp],0)
C:\Users\84678\AppData\Local\Temp\ipykernel_15880\1036127240.py:68: FutureWarning: In a

In [596]:
D1_t = D_t.iloc[:,:2]
D1_t = D1_t.dropna()
D1_t.describe()

,b1,a1
count,274.000000,274.000000
mean,0.794102,0.754346
std,0.225946,0.237537
min,0.111888,0.063180
25%,0.661738,0.598186
50%,0.887924,0.825967
75%,0.977134,0.954955
max,0.999991,0.999993


In [597]:
D1_c = D_c.iloc[:,:2]
D1_c = D1_c.dropna()
D1_c.describe()

,b1,a1
count,173.000000,173.000000
mean,0.812340,0.811994
std,0.219567,0.215658
min,0.119391,0.028841
25%,0.704216,0.696346
50%,0.907752,0.885747
75%,0.966804,0.977852
max,0.999992,0.999999


In [598]:
D2_t = D_t.iloc[:,7:9]
D2_t = D2_t.dropna()
D2_t.describe()

,b2,a2
count,274.000000,274.000000
mean,0.797042,0.764836
std,0.140850,0.150655
min,0.345817,0.291265
25%,0.721080,0.671740
50%,0.812291,0.776914
75%,0.911901,0.888134
max,0.999088,0.999320


In [599]:
D2_c = D_c.iloc[:,7:9]
D2_c = D2_c.dropna()
D2_c.describe()

,b2,a2
count,173.000000,173.000000
mean,0.809662,0.811834
std,0.145273,0.138552
min,0.354860,0.307884
25%,0.735561,0.742480
50%,0.830143,0.833136
75%,0.921567,0.922448
max,0.998684,0.998728


In [600]:
print(ss.ttest_ind(D1_t['a1'],D1_t['b1']))
print(ss.ranksums(D1_t['a1'],D1_t['b1']))

Ttest_indResult(statistic=-2.0073538390019308, pvalue=0.04520430555991408)
RanksumsResult(statistic=-2.2370886053851526, pvalue=0.025280549544998674)


In [601]:
print(ss.ttest_ind(D2_t['a2'],D2_t['b2']))
print(ss.ranksums(D2_t['a2'],D2_t['b2']))

Ttest_indResult(statistic=-2.5848770121775293, pvalue=0.00999961810803556)
RanksumsResult(statistic=-2.688175453938454, pvalue=0.007184363619810551)


In [602]:
print(ss.ttest_ind(D1_c['a1'],D1_c['b1']))
print(ss.ranksums(D1_c['a1'],D1_c['b1']))

Ttest_indResult(statistic=-0.014792134919775346, pvalue=0.988206589467741)
RanksumsResult(statistic=0.24024663306595095, pvalue=0.8101390641165256)


In [603]:
print(ss.ttest_ind(D2_c['a2'],D2_c['b2']))
print(ss.ranksums(D2_c['a2'],D2_c['b2']))

Ttest_indResult(statistic=0.14229600066803977, pvalue=0.8869295087763)
RanksumsResult(statistic=-0.06180841790287329, pvalue=0.9507153997900474)


In [612]:
D_t['makerdum'] = 1
D_c['makerdum'] = 0
D = pd.concat([D_t,D_c],0)
D1 = D.iloc[:,[0,1,2,3,4,5,6,14]]
D2 = D.iloc[:,7:15]
D1 = D1.dropna(how='any')
D2 = D2.dropna(how='any')
D1.to_excel("C:/Users/84678/Desktop/做市商效率/定价效率/D1.xlsx" , index = True)
D2.to_excel("C:/Users/84678/Desktop/做市商效率/定价效率/D2.xlsx" , index = True)

C:\Users\84678\AppData\Local\Temp\ipykernel_15880\4354403.py:3: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  D = pd.concat([D_t,D_c],0)


$pricing = a_0 + a_1Timedum_i + a_2Makerdum_i + a_3Timedum_i×Makerdum_i + a_4Lnprice_i + a_5Lnvolality_i + a_6Lnoutstanding + ε_i$

In [604]:
#倍差法模型
variable_corp = pd.read_excel( 'C:/Users/84678/Desktop/做市商效率/定价效率/did_effi.xlsx',sheet_name = "variable_corp",index_col = 0)
variable_syn = pd.read_excel( 'C:/Users/84678/Desktop/做市商效率/定价效率/did_effi.xlsx',sheet_name = "variable_syn",index_col = 0)
variable_D1 = pd.read_excel( 'C:/Users/84678/Desktop/做市商效率/定价效率/did_effi.xlsx',sheet_name = "variable_D1",index_col = 0)
variable_D2= pd.read_excel( 'C:/Users/84678/Desktop/做市商效率/定价效率/did_effi.xlsx',sheet_name = "variable_D2",index_col = 0)


In [605]:
did_v_c = pd.DataFrame()
did_v_c['lnprice'] = variable_corp['price']
did_v_c['lnvolatility'] = variable_corp['volatility']
did_v_c['lnoutstanding'] = variable_corp['outstanding']
did_v_c['lnmv'] = variable_corp['mv']
did_v_c = zero_out(did_v_c)
did_v_c = np.log(did_v_c)
did_v_c.index = variable_corp.index.values

In [606]:
did_v_s = pd.DataFrame()
did_v_s['lnprice'] = variable_syn['price']
did_v_s['lnvolatility'] = variable_syn['volatility']
did_v_s['lnoutstanding'] = variable_syn['outstanding']
did_v_s['lnmv'] = variable_syn['mv']
did_v_s = zero_out(did_v_s)
did_v_s = np.log(did_v_s)
did_v_s.index = variable_syn.index.values

In [607]:
did_v_c.to_excel("C:/Users/84678/Desktop/做市商效率/定价效率/variable_c.xlsx" , index = True)
did_v_s.to_excel("C:/Users/84678/Desktop/做市商效率/定价效率/variable_s.xlsx" , index = True)

In [608]:
did_v_d1 = pd.DataFrame()
did_v_d1['lnprice'] = variable_D1['price']
did_v_d1['lnvolatility'] = variable_D1['volatility']
did_v_d1['lnoutstanding'] = variable_D1['outstanding']
did_v_d1['lnmv'] = variable_D1['mv']
did_v_d1 = zero_out(did_v_d1)
did_v_d1 = np.log(did_v_d1)
did_v_d1.index = variable_D1.index.values

In [609]:
did_v_d2 = pd.DataFrame()
did_v_d2['lnprice'] = variable_D2['price']
did_v_d2['lnvolatility'] = variable_D2['volatility']
did_v_d2['lnoutstanding'] = variable_D2['outstanding']
did_v_d2['lnmv'] = variable_D2['mv']
did_v_d2 = zero_out(did_v_d2)
did_v_d2= np.log(did_v_d2)
did_v_d2.index = variable_D2.index.values

In [610]:
did_v_d1.to_excel("C:/Users/84678/Desktop/做市商效率/定价效率/variable_d1.xlsx" , index = True)
did_v_d1.to_excel("C:/Users/84678/Desktop/做市商效率/定价效率/variable_d2.xlsx" , index = True)

In [613]:
did_syn = pd.read_excel( 'C:/Users/84678/Desktop/做市商效率/定价效率/did_effi.xlsx',sheet_name = "syn")
did_corp = pd.read_excel( 'C:/Users/84678/Desktop/做市商效率/定价效率/did_effi.xlsx',sheet_name = "corp")

In [614]:
for i in [0,3,4,5,6,7,8]:
    did_syn.iloc[:,i] = winsorize(did_syn.iloc[:,i],limits=[0.03,0.03])
for i in [0,3,4,5,6,7,8]:
    did_corp.iloc[:,i] = winsorize(did_corp.iloc[:,i],limits=[0.03,0.03])

In [615]:
# reg_syn = smf.ols(formula='syn~timedum+makerdum+timedum*makerdum+lnprice+lnvolatility+lnoutstanding',data=did_syn)
reg_syn = smf.ols(formula='syn~timedum+makerdum+timedum*makerdum+lnoutstanding',data=did_syn)
results_syn = reg_syn.fit()
results_syn.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    syn   R-squared:                       0.029
Model:                            OLS   Adj. R-squared:                  0.025
Method:                 Least Squares   F-statistic:                     6.715
Date:                Sat, 08 Jul 2023   Prob (F-statistic):           2.52e-05
Time:                        16:10:14   Log-Likelihood:                -2057.9
No. Observations:                 894   AIC:                             4126.
Df Residuals:                     889   BIC:                             4150.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           -6.4146      0.321    -19.988      0.000      -7.044      -5.785
timedum              0.0925      0.261      0.355      0.723      -0.419       0.604
makerdum            -0.1152      0.236     -0.489      0.625      -0.578       0.347
timedum:makerdum     0.8858      0.333      2.660      0.008       0.232       1.539
lnoutstanding        0.0121      0.018      0.675      0.500      -0.023       0.047
==============================================================================
Omnibus:                      175.261   Durbin-Watson:                   1.927
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              358.663
Skew:                          -1.111   Prob(JB):                     1.31e-78
Kurtosis:                       5.167   Cond. No.                         90.4
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [623]:
reg_corp = smf.ols(formula='corp~timedum+makerdum+timedum*makerdum+lnprice+lnvolatility+lnoutstanding',data=did_corp)
results_corp = reg_corp.fit()
results_corp.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   corp   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.7133
Date:                Sat, 08 Jul 2023   Prob (F-statistic):              0.639
Time:                        16:11:13   Log-Likelihood:                -1376.7
No. Observations:                 804   AIC:                             2767.
Df Residuals:                     797   BIC:                             2800.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           -1.7436      0.228     -7.654      0.000      -2.191      -1.296
timedum              0.1362      0.160      0.852      0.394      -0.178       0.450
makerdum             0.1171      0.141      0.833      0.405      -0.159       0.393
timedum:makerdum    -0.0409      0.199     -0.206      0.837      -0.431       0.349
lnprice              0.0569      0.062      0.918      0.359      -0.065       0.179
lnvolatility         0.0171      0.029      0.593      0.553      -0.040       0.074
lnoutstanding       -0.0087      0.010     -0.842      0.400      -0.029       0.012
==============================================================================
Omnibus:                      127.620   Durbin-Watson:                   2.044
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              272.589
Skew:                          -0.893   Prob(JB):                     6.43e-60
Kurtosis:                       5.224   Cond. No.                         97.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [617]:
did_d1 = pd.read_excel( 'C:/Users/84678/Desktop/做市商效率/定价效率/did_effi.xlsx',sheet_name = "D1")
did_d2 = pd.read_excel( 'C:/Users/84678/Desktop/做市商效率/定价效率/did_effi.xlsx',sheet_name = "D2")

In [618]:
for i in [0,3,4,5,6,7,8]:
    did_d1.iloc[:,i] = winsorize(did_d1.iloc[:,i],limits=[0.03,0.03])
for i in [0,3,4,5,6,7,8]:
    did_d2.iloc[:,i] = winsorize(did_d2.iloc[:,i],limits=[0.03,0.03])

In [628]:
reg_d1 = smf.ols(formula='D1~timedum+makerdum+timedum*makerdum+lnmv+PB+ami',data=did_d1)
results_d1 = reg_d1.fit()
results_d1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     D1   R-squared:                       0.023
Model:                            OLS   Adj. R-squared:                  0.014
Method:                 Least Squares   F-statistic:                     2.583
Date:                Sun, 09 Jul 2023   Prob (F-statistic):             0.0366
Time:                        11:04:04   Log-Likelihood:                 41.304
No. Observations:                 447   AIC:                            -72.61
Df Residuals:                     442   BIC:                            -52.09
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            0.8090      0.038     21.485      0.000       0.735       0.883
timedum            4.75e-17   7.59e-17      0.626      0.532   -1.02e-16    1.97e-16
makerdum            -0.0153      0.022     -0.707      0.480      -0.058       0.027
timedum:makerdum -4.317e-19   2.27e-19     -1.898      0.058   -8.79e-19    1.52e-20
lnmv                 0.0023      0.002      1.209      0.227      -0.001       0.006
PB                  -0.0005      0.001     -0.445      0.656      -0.003       0.002
ami                  0.0059      0.002      2.862      0.004       0.002       0.010
==============================================================================
Omnibus:                       95.930   Durbin-Watson:                   2.051
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              156.509
Skew:                          -1.340   Prob(JB):                     1.03e-34
Kurtosis:                       4.106   Cond. No.                     3.58e+17
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.37e-30. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [629]:
reg_d2 = smf.ols(formula='D2~timedum+makerdum+timedum*makerdum+lnmv+PB+ami',data=did_d2)
results_d2 = reg_d2.fit()
results_d2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     D2   R-squared:                       0.030
Model:                            OLS   Adj. R-squared:                  0.023
Method:                 Least Squares   F-statistic:                     4.529
Date:                Sun, 09 Jul 2023   Prob (F-statistic):           0.000156
Time:                        11:04:19   Log-Likelihood:                 469.43
No. Observations:                 894   AIC:                            -924.9
Df Residuals:                     887   BIC:                            -891.3
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            0.8133      0.019     43.076      0.000       0.776       0.850
timedum              0.0022      0.015      0.141      0.888      -0.028       0.032
makerdum            -0.0114      0.014     -0.814      0.416      -0.039       0.016
timedum:makerdum    -0.0344      0.020     -1.742      0.082      -0.073       0.004
lnmv                 0.0008      0.001      0.973      0.331      -0.001       0.003
PB                  -0.0002      0.001     -0.412      0.680      -0.001       0.001
ami                  0.0030      0.001      3.199      0.001       0.001       0.005
==============================================================================
Omnibus:                       69.292   Durbin-Watson:                   2.032
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               83.899
Skew:                          -0.742   Prob(JB):                     6.05e-19
Kurtosis:                       3.224   Cond. No.                         115.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [54]:
tran_out = pd.read_excel( 'C:/Users/84678/Desktop/做市商效率/退出做市/ex.xlsx',sheet_name = '摘牌退出做市',index_col = 0)

In [112]:
# 1：协议转让，2：竞价交易
tran1 = pd.DataFrame()
tran2 = pd.DataFrame()

temp1 = pd.DataFrame()
temp2 = pd.DataFrame()

time = tran_out.columns.to_frame().T
for j in range(tran_out.shape[0]):
    item = tran_out.iloc[j].to_frame().T
    for i in range(3,item.shape[1]-1):
        if item.iat[0,i] == '做市交易' and item.iat[0,i+1] == '协议转让' :
            temp1['代码'] = item.index
            temp1['时间'] = time.iat[0,i+1]
            tran1 = pd.concat([tran1,temp1],axis = 0)
        if item.iat[0,i] == '做市交易' and item.iat[0,i+1] == '集合竞价交易' :
            temp2['代码'] = item.index
            temp2['时间'] = time.iat[0,i+1]
            tran2 = pd.concat([tran2,temp2],axis = 0)

In [119]:
tran1 = tran1.set_index('代码')
tran1['后续'] = '转协议'

In [121]:
tran2 = tran2.set_index('代码')
tran2['后续'] = '转竞价'

In [122]:
tran2

,时间,后续
代码,,
430029.NQ,2018/12/28,转竞价
430032.NQ,2020/9/9,转竞价
430065.NQ,2018/10/26,转竞价
430074.NQ,2022/8/29,转竞价
430080.NQ,2021/11/15,转竞价
...,...,...
839706.NQ,2018/3/29,转竞价
839951.NQ,2021/1/7,转竞价
839955.NQ,2020/8/25,转竞价


In [123]:
tran = pd.concat([tran1,tran2],axis=0)
tran

,时间,后续
代码,,
430024.NQ,2017/4/21,转协议
430070.NQ,2017/11/16,转协议
430071.NQ,2017/11/13,转协议
430090.NQ,2017/12/14,转协议
430133.NQ,2017/3/9,转协议
...,...,...
839706.NQ,2018/3/29,转竞价
839951.NQ,2021/1/7,转竞价
839955.NQ,2020/8/25,转竞价


In [203]:
num_syn1 = np.log(num_syn1/(1-num_syn1))

In [204]:
num_syn['syn'] = num_syn1['syn']

In [205]:
num_syn = num_syn.set_index('index')

In [207]:
did_num = num_syn.iloc[:,[0,7,8,9,10]]

In [209]:
did_num['lnprice'] = did_num['price']
did_num['lnvolatility'] = did_num['volatility']
did_num['lnoutstanding'] = did_num['outstanding']

C:\Users\84678\AppData\Local\Temp\ipykernel_8708\1953672365.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  did_num['lnprice'] = did_num['price']
C:\Users\84678\AppData\Local\Temp\ipykernel_8708\1953672365.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  did_num['lnvolatility'] = did_num['volatility']
C:\Users\84678\AppData\Local\Temp\ipykernel_8708\1953672365.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

,syn,price,outstanding,volatility,number,lnprice,lnvolatility,lnoutstanding
index,,,,,,,,
430014.NQ,-6.893248,3.5812,63218421.0,26.912324,2,3.5812,26.912324,63218421.0
430021.NQ,-5.605304,21.4067,135298934.0,27.141058,8,21.4067,27.141058,135298934.0
430022.NQ,-4.423941,3.0023,20661000.0,4.627066,5,3.0023,4.627066,20661000.0
430027.NQ,-7.147345,8.5284,21249986.0,9.355758,2,8.5284,9.355758,21249986.0
430046.NQ,-4.022389,19.2731,15671640.0,82.730346,2,19.2731,82.730346,15671640.0
...,...,...,...,...,...,...,...,...
835553.NQ,-6.561471,8.7123,0.0,2.605143,2,8.7123,2.605143,0.0
835692.NQ,-3.489328,4.4982,1755000.0,1.953339,4,4.4982,1.953339,1755000.0
836030.NQ,-3.249524,3.0792,3995500.0,1.440915,3,3.0792,1.440915,3995500.0


In [212]:
did_num.iloc[:,[5,6,7]] = zero_out(did_num.iloc[:,[5,6,7]])
did_num.iloc[:,[5,6,7]] = np.log(did_num.iloc[:,[5,6,7]])

C:\Users\84678\AppData\Local\Temp\ipykernel_8708\2429489621.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.iloc[i,j] = 1
C:\Users\84678\AppData\Local\Temp\ipykernel_8708\370090427.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  did_num.iloc[:,[5,6,7]] = zero_out(did_num.iloc[:,[5,6,7]])
C:\Users\84678\AppData\Local\Temp\ipykernel_8708\370090427.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.py

In [213]:
did_num

,syn,price,outstanding,volatility,number,lnprice,lnvolatility,lnoutstanding
index,,,,,,,,
430014.NQ,-6.893248,3.5812,63218421.0,26.912324,2,1.275698,3.292584,17.962106
430021.NQ,-5.605304,21.4067,135298934.0,27.141058,8,3.063704,3.301048,18.722997
430022.NQ,-4.423941,3.0023,20661000.0,4.627066,5,1.099379,1.531923,16.843758
430027.NQ,-7.147345,8.5284,21249986.0,9.355758,2,2.143402,2.235992,16.871867
430046.NQ,-4.022389,19.2731,15671640.0,82.730346,2,2.958710,4.415586,16.567363
...,...,...,...,...,...,...,...,...
835553.NQ,-6.561471,8.7123,0.0,2.605143,2,2.164736,0.957488,0.000000
835692.NQ,-3.489328,4.4982,1755000.0,1.953339,4,1.503677,0.669540,14.377979
836030.NQ,-3.249524,3.0792,3995500.0,1.440915,3,1.124670,0.365278,15.200679


In [214]:
for i in [0,5,6,7]:
    did_num.iloc[:,i] = winsorize(did_num.iloc[:,i],limits=[0.03,0.03])

C:\Users\84678\AppData\Local\Temp\ipykernel_8708\3221168912.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  did_num.iloc[:,i] = winsorize(did_num.iloc[:,i],limits=[0.03,0.03])


In [216]:
reg_num_s = smf.ols(formula='syn~number+lnprice+lnvolatility+lnoutstanding',data=did_num)
results_num_s = reg_num_s.fit()
results_num_s.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    syn   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     1.440
Date:                Tue, 13 Jun 2023   Prob (F-statistic):              0.219
Time:                        22:57:42   Log-Likelihood:                -1157.2
No. Observations:                 532   AIC:                             2324.
Df Residuals:                     527   BIC:                             2346.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -6.3780      0.490    -13.006      0.000      -7.341      -5.415
number           -0.0253      0.036     -0.699      0.485      -0.096       0.046
lnprice           0.1331      0.130      1.023      0.307      -0.122       0.388
lnvolatility      0.1183      0.068      1.749      0.081      -0.015       0.251
lnoutstanding     0.0279      0.023      1.213      0.226      -0.017       0.073
==============================================================================
Omnibus:                       33.944   Durbin-Watson:                   2.017
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               38.832
Skew:                          -0.654   Prob(JB):                     3.70e-09
Kurtosis:                       3.198   Cond. No.                         89.4
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [338]:
reg_num_c = smf.ols(formula='corp~number+lnprice+lnvolatility+lnoutstanding',data=num_corp)
results_num_c = reg_num_c.fit()
results_num_c.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   corp   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                 -0.004
Method:                 Least Squares   F-statistic:                    0.5568
Date:                Sun, 11 Jun 2023   Prob (F-statistic):              0.694
Time:                        22:22:25   Log-Likelihood:                -886.15
No. Observations:                 506   AIC:                             1782.
Df Residuals:                     501   BIC:                             1803.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -1.3358      0.264     -5.066      0.000      -1.854      -0.818
number           -0.0297      0.027     -1.106      0.269      -0.082       0.023
lnprice           0.0193      0.079      0.246      0.806      -0.135       0.174
lnvolatility     -0.0252      0.035     -0.722      0.471      -0.094       0.043
lnoutstanding    -0.0087      0.012     -0.715      0.475      -0.033       0.015
==============================================================================
Omnibus:                       98.894   Durbin-Watson:                   1.988
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              207.623
Skew:                          -1.053   Prob(JB):                     8.23e-46
Kurtosis:                       5.327   Cond. No.                         68.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Christie and Schultz(1994) 价格双峰分布 隐形合谋假说

In [1]:
# import psmatching.match as psm
# import pytest
# from psmatching.utilities import *
# from sklearn.linear_model import LogisticRegression as LR

In [9]:
path_c = 'C:/Users/84678/Desktop/做市商效率/PSM/数据处理/新三板竞价企业.xlsx'
df_n_c_assets = pd.read_excel(path_c, sheet_name = "总资产",index_col = 0)
df_n_c_close = pd.read_excel(path_c, sheet_name = "收盘价",index_col = 0)
df_n_c_tradedays = pd.read_excel(path_c, sheet_name = "交易天数",index_col = 0)
df_n_c_share = pd.read_excel(path_c, sheet_name = "流通股合计",index_col = 0)
df_n_c_condition = pd.read_excel(path_c, sheet_name = "交易状态",index_col = 0)
df_n_c_roa = pd.read_excel(path_c, sheet_name = "ROA",index_col = 0)
df_n_c_le = pd.read_excel(path_c, sheet_name = "资产负债率",index_col = 0)
df_n_c_top = pd.read_excel(path_c, sheet_name = "TOP1",index_col = 0)

In [2]:
cir_list = [['asset'],
 ['meanprice'],
 ['day'],
 ['outstanding'],
 ['roa'],
 ['leverage'],
 ['top'],
 ['asset', 'meanprice'],
 ['asset', 'day'],
 ['asset', 'outstanding'],
 ['asset', 'roa'],
 ['asset', 'leverage'],
 ['asset', 'top'],
 ['meanprice', 'day'],
 ['meanprice', 'outstanding'],
 ['meanprice', 'roa'],
 ['meanprice', 'leverage'],
 ['meanprice', 'top'],
 ['day', 'outstanding'],
 ['day', 'roa'],
 ['day', 'leverage'],
 ['day', 'top'],
 ['outstanding', 'roa'],
 ['outstanding', 'leverage'],
 ['outstanding', 'top'],
 ['roa', 'leverage'],
 ['roa', 'top'],
 ['leverage', 'top'],
 ['asset', 'meanprice', 'day'],
 ['asset', 'meanprice', 'outstanding'],
 ['asset', 'meanprice', 'roa'],
 ['asset', 'meanprice', 'leverage'],
 ['asset', 'meanprice', 'top'],
 ['asset', 'day', 'outstanding'],
 ['asset', 'day', 'roa'],
 ['asset', 'day', 'leverage'],
 ['asset', 'day', 'top'],
 ['asset', 'outstanding', 'roa'],
 ['asset', 'outstanding', 'leverage'],
 ['asset', 'outstanding', 'top'],
 ['asset', 'roa', 'leverage'],
 ['asset', 'roa', 'top'],
 ['asset', 'leverage', 'top'],
 ['meanprice', 'day', 'outstanding'],
 ['meanprice', 'day', 'roa'],
 ['meanprice', 'day', 'leverage'],
 ['meanprice', 'day', 'top'],
 ['meanprice', 'outstanding', 'roa'],
 ['meanprice', 'outstanding', 'leverage'],
 ['meanprice', 'outstanding', 'top'],
 ['meanprice', 'roa', 'leverage'],
 ['meanprice', 'roa', 'top'],
 ['meanprice', 'leverage', 'top'],
 ['day', 'outstanding', 'roa'],
 ['day', 'outstanding', 'leverage'],
 ['day', 'outstanding', 'top'],
 ['day', 'roa', 'leverage'],
 ['day', 'roa', 'top'],
 ['day', 'leverage', 'top'],
 ['outstanding', 'roa', 'leverage'],
 ['outstanding', 'roa', 'top'],
 ['outstanding', 'leverage', 'top'],
 ['roa', 'leverage', 'top'],
 ['asset', 'meanprice', 'day', 'outstanding'],
 ['asset', 'meanprice', 'day', 'roa'],
 ['asset', 'meanprice', 'day', 'leverage'],
 ['asset', 'meanprice', 'day', 'top'],
 ['asset', 'meanprice', 'outstanding', 'roa'],
 ['asset', 'meanprice', 'outstanding', 'leverage'],
 ['asset', 'meanprice', 'outstanding', 'top'],
 ['asset', 'meanprice', 'roa', 'leverage'],
 ['asset', 'meanprice', 'roa', 'top'],
 ['asset', 'meanprice', 'leverage', 'top'],
 ['asset', 'day', 'outstanding', 'roa'],
 ['asset', 'day', 'outstanding', 'leverage'],
 ['asset', 'day', 'outstanding', 'top'],
 ['asset', 'day', 'roa', 'leverage'],
 ['asset', 'day', 'roa', 'top'],
 ['asset', 'day', 'leverage', 'top'],
 ['asset', 'outstanding', 'roa', 'leverage'],
 ['asset', 'outstanding', 'roa', 'top'],
 ['asset', 'outstanding', 'leverage', 'top'],
 ['asset', 'roa', 'leverage', 'top'],
 ['meanprice', 'day', 'outstanding', 'roa'],
 ['meanprice', 'day', 'outstanding', 'leverage'],
 ['meanprice', 'day', 'outstanding', 'top'],
 ['meanprice', 'day', 'roa', 'leverage'],
 ['meanprice', 'day', 'roa', 'top'],
 ['meanprice', 'day', 'leverage', 'top'],
 ['meanprice', 'outstanding', 'roa', 'leverage'],
 ['meanprice', 'outstanding', 'roa', 'top'],
 ['meanprice', 'outstanding', 'leverage', 'top'],
 ['meanprice', 'roa', 'leverage', 'top'],
 ['day', 'outstanding', 'roa', 'leverage'],
 ['day', 'outstanding', 'roa', 'top'],
 ['day', 'outstanding', 'leverage', 'top'],
 ['day', 'roa', 'leverage', 'top'],
 ['outstanding', 'roa', 'leverage', 'top'],
 ['asset', 'meanprice', 'day', 'outstanding', 'roa'],
 ['asset', 'meanprice', 'day', 'outstanding', 'leverage'],
 ['asset', 'meanprice', 'day', 'outstanding', 'top'],
 ['asset', 'meanprice', 'day', 'roa', 'leverage'],
 ['asset', 'meanprice', 'day', 'roa', 'top'],
 ['asset', 'meanprice', 'day', 'leverage', 'top'],
 ['asset', 'meanprice', 'outstanding', 'roa', 'leverage'],
 ['asset', 'meanprice', 'outstanding', 'roa', 'top'],
 ['asset', 'meanprice', 'outstanding', 'leverage', 'top'],
 ['asset', 'meanprice', 'roa', 'leverage', 'top'],
 ['asset', 'day', 'outstanding', 'roa', 'leverage'],
 ['asset', 'day', 'outstanding', 'roa', 'top'],
 ['asset', 'day', 'outstanding', 'leverage', 'top'],
 ['asset', 'day', 'roa', 'leverage', 'top'],
 ['asset', 'outstanding', 'roa', 'leverage', 'top'],
 ['meanprice', 'day', 'outstanding', 'roa', 'leverage'],
 ['meanprice', 'day', 'outstanding', 'roa', 'top'],
 ['meanprice', 'day', 'outstanding', 'leverage', 'top'],
 ['meanprice', 'day', 'roa', 'leverage', 'top'],
 ['meanprice', 'outstanding', 'roa', 'leverage', 'top'],
 ['day', 'outstanding', 'roa', 'leverage', 'top'],
 ['asset', 'meanprice', 'day', 'outstanding', 'roa', 'leverage'],
 ['asset', 'meanprice', 'day', 'outstanding', 'roa', 'top'],
 ['asset', 'meanprice', 'day', 'outstanding', 'leverage', 'top'],
 ['asset', 'meanprice', 'day', 'roa', 'leverage', 'top'],
 ['asset', 'meanprice', 'outstanding', 'roa', 'leverage', 'top'],
 ['asset', 'day', 'outstanding', 'roa', 'leverage', 'top'],
 ['meanprice', 'day', 'outstanding', 'roa', 'leverage', 'top'],
 ['asset', 'meanprice', 'day', 'outstanding', 'roa', 'leverage', 'top']]

In [10]:
# output_DF = []
# for cir_n in range(127):
#     df_compare = pd.DataFrame()
#     df_match = pd.DataFrame()

#     #排除做市日前后200天内有停牌的公司
#     for i in range(df_n_m.shape[0]):
#         vari_c = pd.DataFrame()
#         vari_t = pd.DataFrame()
#         vari = pd.DataFrame()

#         df_psm = pd.DataFrame()
#         r = df_n_m.iloc[i]
#         date_b1 = r['做市前200交易日']
#         date_b2 = r['做市前100交易日']
#         date_b3 = r['做市交易前一日']
#         date_a1 = r['做市首日']
#         date_a2 = r['做市后200交易日']
#         date_c = r['匹配期']
#         temp = df_n_c_condition.loc[:,date_b1:date_a2]
#         temp = temp.dropna(how = 'any')

#         for j in range(temp.shape[0]):
#             if '停牌一天' in temp.iloc[j]:
#                 temp = temp.drop[j]

#             #对每个做市交易的公司只选择temp内的竞价交易的企业
#         list_index = temp.index.values
#         df_n_c_assets1 = df_n_c_assets.loc[list_index]
#         df_n_c_close1 = df_n_c_close.loc[list_index] 
#         df_n_c_tradedays1 = df_n_c_tradedays.loc[list_index] 
#         df_n_c_share1 = df_n_c_share.loc[list_index]
#         df_n_c_roa1 = df_n_c_roa.loc[list_index]
#         df_n_c_le1 = df_n_c_le.loc[list_index]
#         df_n_c_top1 = df_n_c_top.loc[list_index]

#         #计算区间内平均收盘价，交易天数和匹配期的总资产和流通股合计
#         close = (df_n_c_close1.loc[ : , date_b2:date_b3]).mean(1)
#         tradedays_temp = (df_n_c_tradedays1.loc[:,date_b2:date_b3]).fillna(0)
#         tradedays = tradedays_temp.gt(0).sum(axis=1)
#         assets = df_n_c_assets1.loc[ : , date_c]
#         share = df_n_c_share1.loc[ : , date_c]
#         roa = df_n_c_roa1.loc[ : , date_c]
#         le = df_n_c_le1.loc[ : , date_c]
#         top = df_n_c_top1.loc[ : , date_c]

#         #将竞价企业数据集合在一个表中
#         df_n_c = pd.concat([assets,close,tradedays,share,roa,le,top],axis = 1,ignore_index = True)
#         df_n_c.columns = ['asset','meanprice','day','outstanding','roa','leverage','top']
#         df_n_c = df_n_c.drop(df_n_c.loc[(df_n_c['asset']==0)].index)
#         df_n_c = df_n_c.drop(df_n_c.loc[(df_n_c['meanprice']==0) & (df_n_c['day']==0) & (df_n_c['outstanding'] ==0)].index)
#         df_n_c = df_n_c.drop(df_n_c.loc[(df_n_c['roa']<=0)].index)

#         df_n_c['treated'] = 0
#         r_temp = r.to_frame().T
#         r_temp = r_temp.iloc[:,6:]
#         r_temp['treated'] = 1
#         df_psm = pd.concat([df_n_c,r_temp],axis = 0)

#         df_psm['asset'] = pd.to_numeric(df_psm['asset'])
#         df_psm['meanprice'] = pd.to_numeric(df_psm['meanprice'])
#         df_psm['day'] = pd.to_numeric(df_psm['day'])
#         df_psm['outstanding'] = pd.to_numeric(df_psm['outstanding'])
#         df_psm['roa'] = pd.to_numeric(df_psm['roa'])
#         df_psm['leverage'] = pd.to_numeric(df_psm['leverage'])
#         df_psm['top'] = pd.to_numeric(df_psm['top'])

#         y_samp = df_psm['treated']
#         x_samp = df_psm[cir_list[cir_n]]


#         logreg = LR()
#         logreg.fit(x_samp, y_samp)

#         # 预测倾向得分
#         propensity_scores = logreg.predict_proba(x_samp)[:, 1]

#         df_psm["PROPENSITY"] = propensity_scores
#         groups = df_psm.treated
#         propensity = df_psm.PROPENSITY
#         g1, g2 = propensity[groups==1], propensity[groups==0]

#         test_scores = df_psm[df_psm['treated']==1][['PROPENSITY']]
#         ctrl_scores = df_psm[df_psm['treated']==0][['PROPENSITY']]
#         ctrl_scores ['diff'] = abs(ctrl_scores - test_scores.iat[0,0])
#         match = ctrl_scores.sort_values('diff').head(1)

#         match_temp = pd.concat([df_psm.loc[test_scores.index.values],df_psm.loc[match.index.values]],axis=0)

#         match_temp = match_temp.iloc[:,:8]

#         treatment = match_temp[match_temp['treated']==1]
#         control = match_temp[match_temp['treated']==0]

#         control = control.drop_duplicates()

#         control['index'] = treatment.index.values
#         control = control.reset_index()
#         control  = control.set_index('index')
#         control.columns=['index_c','asset_c','meanprice_c','day_c','outstanding_c','roa_c','leverage_c','top_c','treated_c']
#         matches = pd.concat([treatment,control],axis=1)
#         df_match = pd.concat([df_match,matches],axis=0)
#     temp = []
#     for i in range(7):    
#         temp.append(ss.ttest_ind(df_match.iloc[:,i],df_match.iloc[:,i+9])[1])
#     output_DF.append(temp)
#     # (pd.DataFrame(output_DF)).to_excel('cir_res.xlsx')
#     if cir_n%10 == 0 or cir_n == 126:
#         (pd.DataFrame(output_DF)).to_excel('cir_res.xlsx')
#         print(f'save {cir_n+1}th file.')

In [18]:
# df_match = pd.DataFrame()
# #排除做市日前后200天内有停牌的公司
# for i in range(df_n_m.shape[0]):
#     vari_c = pd.DataFrame()
#     vari_t = pd.DataFrame()
#     vari = pd.DataFrame()

#     df_psm = pd.DataFrame()
#     r = df_n_m.iloc[i]
#     date_b1 = r['做市前200交易日']
#     date_b2 = r['做市前100交易日']
#     date_b3 = r['做市交易前一日']
#     date_a1 = r['做市首日']
#     date_a2 = r['做市后200交易日']
#     date_c = r['匹配期']
#     temp = df_n_c_condition.loc[:,date_b1:date_a2]
#     temp = temp.dropna(how = 'any')

#     for j in range(temp.shape[0]):
#         if '停牌一天' in temp.iloc[j]:
#             temp = temp.drop[j]

#         #对每个做市交易的公司只选择temp内的竞价交易的企业
#     list_index = temp.index.values
#     df_n_c_assets1 = df_n_c_assets.loc[list_index]
#     df_n_c_close1 = df_n_c_close.loc[list_index] 
#     df_n_c_tradedays1 = df_n_c_tradedays.loc[list_index] 
#     df_n_c_share1 = df_n_c_share.loc[list_index]
#     df_n_c_roa1 = df_n_c_roa.loc[list_index]
#     df_n_c_le1 = df_n_c_le.loc[list_index]
#     df_n_c_top1 = df_n_c_top.loc[list_index]

#     #计算区间内平均收盘价，交易天数和匹配期的总资产和流通股合计
#     close = (df_n_c_close1.loc[ : , date_b2:date_b3]).mean(1)
#     tradedays_temp = (df_n_c_tradedays1.loc[:,date_b2:date_b3]).fillna(0)
#     tradedays = tradedays_temp.gt(0).sum(axis=1)
#     assets = df_n_c_assets1.loc[ : , date_c]
#     share = df_n_c_share1.loc[ : , date_c]
#     roa = df_n_c_roa1.loc[ : , date_c]
#     le = df_n_c_le1.loc[ : , date_c]
#     top = df_n_c_top1.loc[ : , date_c]

#     #将竞价企业数据集合在一个表中
#     df_n_c = pd.concat([assets,close,tradedays,share,roa,le,top],axis = 1,ignore_index = True)
#     df_n_c.columns = ['asset','meanprice','day','outstanding','roa','leverage','top']
#     df_n_c = df_n_c.drop(df_n_c.loc[(df_n_c['asset']==0)].index)
#     df_n_c = df_n_c.drop(df_n_c.loc[(df_n_c['meanprice']==0) & (df_n_c['day']==0) & (df_n_c['outstanding'] ==0)].index)
#     df_n_c = df_n_c.drop(df_n_c.loc[(df_n_c['roa']<=0)].index)

#     df_n_c['treated'] = 0
#     r_temp = r.to_frame().T
#     r_temp = r_temp.iloc[:,6:]
#     r_temp['treated'] = 1
#     df_psm = pd.concat([df_n_c,r_temp],axis = 0)

#     df_psm['asset'] = pd.to_numeric(df_psm['asset'])
#     df_psm['meanprice'] = pd.to_numeric(df_psm['meanprice'])
#     df_psm['day'] = pd.to_numeric(df_psm['day'])
#     df_psm['outstanding'] = pd.to_numeric(df_psm['outstanding'])
#     df_psm['roa'] = pd.to_numeric(df_psm['roa'])
#     df_psm['leverage'] = pd.to_numeric(df_psm['leverage'])
#     df_psm['top'] = pd.to_numeric(df_psm['top'])

#     y_samp = df_psm['treated']
#     x_samp = df_psm[['meanprice','day','top']]


#     logreg = LR()
#     logreg.fit(x_samp, y_samp)

#     # 预测倾向得分
#     propensity_scores = logreg.predict_proba(x_samp)[:, 1]

#     df_psm["PROPENSITY"] = propensity_scores
#     groups = df_psm.treated
#     propensity = df_psm.PROPENSITY
#     g1, g2 = propensity[groups==1], propensity[groups==0]

#     test_scores = df_psm[df_psm['treated']==1][['PROPENSITY']]
#     ctrl_scores = df_psm[df_psm['treated']==0][['PROPENSITY']]
#     ctrl_scores ['diff'] = abs(ctrl_scores - test_scores.iat[0,0])
#     match = ctrl_scores.sort_values('diff').head(1)

#     match_temp = pd.concat([df_psm.loc[test_scores.index.values],df_psm.loc[match.index.values]],axis=0)

#     match_temp = match_temp.iloc[:,:8]

#     treatment = match_temp[match_temp['treated']==1]
#     control = match_temp[match_temp['treated']==0]

#     control = control.drop_duplicates()

#     control['index'] = treatment.index.values
#     control = control.reset_index()
#     control  = control.set_index('index')
#     control.columns=['index_c','asset_c','meanprice_c','day_c','outstanding_c','roa_c','leverage_c','top_c','treated_c']
#     matches = pd.concat([treatment,control],axis=1)
#     df_match = pd.concat([df_match,matches],axis=0)


# 科创板

In [ ]:
week = pd.read_excel( 'C:/Users/84678/Desktop/做市商效率/定价效率/df_effi.xlsx',sheet_name = "week")